In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import json
import pprint
import pandas as pd
from train_dict import get_hyperparameters, save_hyperparameters, load_dictionary, get_dictionary_paths
from data import get_original_datasets, print_gpu_memory_usage, print_memory_usage, get_n_classes, get_interpret_objs
from train import SparsePLMICDClassifierModule
from dict_att_pretrained import load_plm_icd
from pyhealth.metrics import multilabel_metrics_fn

# L1 

In [ ]:
dataset = "mimiciii_clean"
n_classes = get_n_classes(dataset)
finetune_batchsize = 8
gpu = 0
device = torch.device(f"cuda:{gpu}")
device = torch.device("cpu")
use_finetuned_model = True
freeze_plm = False 
projection_size = 768 
recon_lambda = 1e-6
lr = 5e-5
dictionary_type = "L1"
plm_path="RoBERTa-base-PM-M3-Voc-hf"
alpha = 8
d_mlp = 768
num_epochs = 20
run_id = 0
classifier_head_type = "dma"

In [ ]:
def load_dma_experiment(run_id, 
                        dataset,
                        dictionary_type,
                        device, 
                        projection_size = 768, 
                        lr = 5e-5, 
                        recon_lambda = 1e-6, 
                        alpha = 8,
                        use_finetuned_model = True,
                        classifier_head_type = "dma", 
                        freeze_plm=False,
                        freeze_dict=False,
                        use_lr_scheduler=True, 
                        num_epochs=20, 
                        d_mlp=768):
    n_classes = get_n_classes(dataset)
    experiments_directory = os.path.join("experiments", "dict_att", dataset)
    chkpts_directory = os.path.join(experiments_directory, "chkpts", classifier_head_type)
    plm_path = "RoBERTa-base-PM-M3-Voc-hf"
    chkpt_name = f"{classifier_head_type}_dict_{dictionary_type}_att_lr_{lr}_hidden_size_{projection_size}_rl{recon_lambda}_E{num_epochs}_ft{use_finetuned_model}_freeze_{freeze_plm}_R{run_id}.ckpt"
    chkpt_path = os.path.join(chkpts_directory, chkpt_name)
    plm_icd = load_plm_icd(plm_path, dataset,  n_classes, device, use_finetuned_model)
    plm = plm_icd.roberta
    label_directory = os.path.join("experiments", "dict_att", dataset, "label_embeddings")
    label_emb_path = os.path.join(label_directory, f"{dictionary_type}_ft_{use_finetuned_model}_icd_sparse_emb_mean.pt")

    dictionary, params = load_dictionary(dataset, dictionary_type="L1", alpha=alpha, finetuned_plm=use_finetuned_model)
    alpha = params["alpha"]
    dictionary_size = alpha * d_mlp
    model = SparsePLMICDClassifierModule.load_from_checkpoint(
        chkpt_path,
        plm=plm,
        classifier_head_type=classifier_head_type, 
        dictionary=dictionary, 
        dictionary_size = dictionary_size, 
        projection_size= projection_size, 
        hidden_size=d_mlp,
        num_classes = n_classes,
        dataset= dataset, 
        lr= lr,
        recon_lambda=recon_lambda,
        freeze_plm = freeze_plm,
        freeze_dictionary = freeze_dict,
        use_scheduler= use_lr_scheduler,
        path_to_icd_memory= label_emb_path, 
        map_location=device
    )
    return model

In [ ]:
def load_all_summaries(baseline, topk=4):
    path = f"experiments/dict_att/mimiciii_clean/dictionary/{baseline}_feature_summaries_top{topk}.json" 
    with open(path, "r") as f:
        return json.load(f)

def reduce_baseline(contexts_dictionary):
    for fid, context_dict in contexts_dictionary.items():
        for i in range(len(context_dict["context_string"])):
            context = context_dict["context_string"][i]
            f_acts = context_dict["context_f_acts"][i]
            tokens = context_dict["context_tokens"][i]
            # get nonzero indices
            values, indices = torch.sort(f_acts, descending=True)
            topk_indices = indices[:5] # top 5
            print(topk_indices)
            # sort those in ascending order to maintain order
            topk_indices = topk_indices.sort()[0]
            print(topk_indices)
            new_context_strings = []
            new_context_tokens = []
            new_f_acts = []
            for idx in topk_indices:
                new_context_strings.append(context[idx])
                new_context_tokens.append(tokens[idx])
                new_f_acts.append(f_acts[idx])
            context_dict["context_string"][i] = new_context_strings
            context_dict["context_tokens"][i] = new_context_tokens
            context_dict["context_f_acts"][i] = new_f_acts
            print(context_dict["context_string"][i])
    return contexts_dictionary  

L1_summaries = load_all_summaries("L1")
RELU_summaries = load_all_summaries("RELU_REDUCED")
L1_contexts = load_dict_contexts_fast(dataset=dataset, batch_size=64, n_batches=None)
L1_reduced_contexts = load_baseline_contexts_fast(dataset=dataset, batch_size=64, n_batches=None, baseline="L1_REDUCED")
RELU_clean_contexts = load_baseline_contexts_fast(dataset=dataset, batch_size=16, n_batches=None, baseline="RELU_CLEAN")
RELU_reduced = reduce_baseline(RELU_clean_contexts)

In [ ]:
model = load_dma_experiment(run_id, dataset, dictionary_type, device)

In [ ]:
global_mat = model.classifier_head.icd_memory.data.squeeze()
print(global_mat.shape)

def get_contexts(contexts_dictionary,fid, topk):
    ctxt_dct = contexts_dictionary[fid]
    context_strings = ctxt_dct["context_string"]
    return context_strings[:topk]    

def extract_relevant_dictionary_features_per_ICD(global_matrix, icd_idx : int, label_transform, topk=10):
    print("For ICD:", label_transform.index2target[int(icd_idx)])
    relevant_f_vec = global_matrix[icd_idx].squeeze()
    # sort
    val, indices = torch.sort(relevant_f_vec, descending=True)
    return indices[:topk], val[:topk] 

    # return indices[:topk], val[:topk]

In [ ]:
test_loader, text_encoder, label_transform, text_transform = get_interpret_objs(dataset)

In [ ]:
top_ICDs = global_mat.max(dim=1).values
top_ICDs, top_ICD_indices = torch.sort(top_ICDs, descending=True)
label_transform.index2target[top_ICD_indices[0].item()]

In [ ]:
top_fs = global_mat.max(dim=0).values
top_fs, top_f_idxs = torch.sort(top_fs, descending=True)
print(top_f_idxs[:5])
print(top_fs[:5])
print(global_mat.max())

In [ ]:
import random
from pyhealth.medcode import InnerMap

# for each ICD code, extract topk dictionary features and their contexts
def get_top_ICD_codes(global_mat, contexts, contexts_summary, label_transform, n, topk):
    ICD_dict = {}
    max_icd_codes = global_mat.max(dim=1).values
    print(max_icd_codes.shape)
    max_icd_codes, icd_idxs = torch.sort(max_icd_codes, descending=True)
    for icd_idx in icd_idxs[:n]:
        icd_code = label_transform.index2target[icd_idx.item()]
        print("ICD:", icd_code)
        icd9cm = InnerMap.load("ICD9CM")
        if icd_code in icd9cm:
            print(icd9cm.lookup(icd_code))
            top_f_idxs, top_f_values = extract_relevant_dictionary_features_per_ICD(global_mat, icd_idx, label_transform, topk=topk)
            print("Top Features:")
            print(top_f_idxs)
            print(top_f_values)
            for j in range(topk):
                if top_f_idxs[j].item() in contexts:
                    # context_str = get_contexts(contexts, top_f_idxs[j].item(), 4)
                    # print(context_str)
                    if str(top_f_idxs[j].item()) in contexts_summary:
                        print(contexts_summary[str(top_f_idxs[j].item())])
                    
            print("\n\n")


# for each ICD code, extract topk dictionary features and their contexts
def random_sample_ICD_codes(global_mat, contexts, contexts_summary, label_transform, n, topk):
    ICD_dict = {}
    icd_idxs = random.sample(range(global_mat.shape[0]), n)
    for icd_idx in icd_idxs:
        icd_code = label_transform.index2target[icd_idx]
        print("ICD:", icd_code)
        icd9cm = InnerMap.load("ICD9CM")
        if icd_code in icd9cm:
            print(icd9cm.lookup(icd_code))
        top_f_idxs, top_f_values = extract_relevant_dictionary_features_per_ICD(global_mat, icd_idx, label_transform, topk=topk)
        print("Top Features:")
        print(top_f_idxs)
        print(top_f_values)
        for j in range(topk):
            if top_f_idxs[j].item() in contexts:
                context_str = get_contexts(contexts, top_f_idxs[j].item(), 4)
                print(context_str)
                if str(top_f_idxs[j].item()) in contexts_summary:
                    print(contexts_summary[str(top_f_idxs[j].item())])
                
        print("\n\n")

# random_sample_ICD_codes(global_mat, L1_reduced_contexts, L1_summaries, label_transform, 25, 6)
get_top_ICD_codes(global_mat, L1_reduced_contexts, L1_summaries, label_transform, 25, 30)

In [ ]:
extract_relevant_dictionary_features_per_ICD(global_mat, 3545, label_transform)

# note to self, ICD codes in the class predicted vector are not ordered, also looking at top ICD codes and their dictionary features, can be hard to discern ICD code from multiple features.

In [ ]:
# For a feature index, extract the top 5 ICD Codes and 4 other least relevant ICD codes
def extract_ICD_codes_per_feature(global_matrix, feature_idx : int, label_transform, topk=5):
    ret_dict = {}
    print("For Feature:", feature_idx)
    relevant_icd_vec = global_matrix[:, feature_idx].squeeze()
    # sort
    print(relevant_icd_vec.shape)
    val, indices = torch.sort(relevant_icd_vec, descending=True)
    ret_dict["top"] = []
    ret_dict["top_idx"] = []
    ret_dict["top_val"] = []
    for i in range(0, topk):
        ret_dict["top"].append(label_transform.index2target[int(indices[i].item())])
        ret_dict["top_idx"].append(indices[i].item())
        ret_dict["top_val"].append(val[i].item())
    ret_dict["least"] = []
    ret_dict["least_idx"] = []
    ret_dict["least_val"] = []
    for i in range(1, 6):
        ret_dict["least"].append(label_transform.index2target[int(indices[-i])])
        ret_dict["least_idx"].append(indices[-i].item())
        ret_dict["least_val"].append(val[-i].item()) 
    return ret_dict

fid = 5861
ICD_Codes= extract_ICD_codes_per_feature(global_mat, fid, label_transform,topk=10)
top = ICD_Codes["top"]
least = ICD_Codes["least"]
print(ICD_Codes)
print(get_contexts(L1_reduced_contexts, fid, 5))
# print(L1_reduced_contexts)
# print(L1_summaries.keys())
print(L1_summaries[str(fid)])
from pyhealth.medcode import InnerMap
icd9cm = InnerMap.load("ICD9CM")
for icd in top:
    if icd in icd9cm:
        print(icd9cm.lookup(icd))
# print(icd9cm.lookup(top[0]))
# print(icd9cm.lookup(top[1]))
# print(icd9cm.lookup(least[0]))


In [ ]:
# let's look at top dictionary features from our matrix
def get_top_dictionary_features(global_mat, contexts_summary, contexts, n, topk):
    max_dictionary_features = global_mat.max(dim=0).values
    max_f_vals, max_f_idxs = torch.sort(max_dictionary_features, descending=True)
    max_f_idxs = max_f_idxs[:n]
    icd9cm = InnerMap.load("ICD9CM")
    icd9proc = InnerMap.load("ICD9PROC")
    for idx in max_f_idxs:
        if idx.item() in contexts:
            context_str = get_contexts(contexts, idx.item(), 4)
            # print(context_str)
            if str(idx.item()) in contexts_summary and len(context_str) > 3:
                print("Feature:", idx)
                top_ICD_codes = extract_ICD_codes_per_feature(global_mat, idx, label_transform, topk)
                print(contexts_summary[str(idx.item())])
                print(context_str)
                for icd in top_ICD_codes["top"]:
                    icd_desc = ""
                    if icd in icd9cm:
                        icd_desc = icd9cm.lookup(icd)
                    elif icd in icd9proc:
                        icd_desc = icd9proc.lookup(icd)
                    print(icd_desc)
                print("\n\n")

def get_random_dictionary_features(global_mat, contexts_summary, contexts, n, topk):
    random_f_idxs = torch.randperm(global_mat.shape[1])[:n]
    icd9cm = InnerMap.load("ICD9CM")
    icd9proc = InnerMap.load("ICD9PROC")
    for idx in random_f_idxs:
        if idx.item() in contexts:
            context_str = get_contexts(contexts, idx.item(), 4)
            # print(context_str)
            if str(idx.item()) in contexts_summary and len(context_str) > 3:
                print("Feature:", idx)
                top_ICD_codes = extract_ICD_codes_per_feature(global_mat, idx, label_transform, topk)
                print(contexts_summary[str(idx.item())])
                print(context_str)
                for icd in top_ICD_codes["top"]:
                    print(icd)
                    if icd in icd9cm:
                        print(icd9cm.lookup(icd))
                    elif icd in icd9proc:
                        print(icd9proc.lookup(icd))
                print("\n\n")

# returns n_random chunks of k ICD codes that are unrelated to the feature based on the matrix and ICD codes
# i.e not in the original set of ICD codes
# low magnitude of global_mat weights
def get_random_unrelated_ICD_codes(f_idx, global_mat, label_transform, ICD_codes, n_random, k):
    random_chunks = []
    random_chunks_desc = []
    ICD_vec = global_mat[:, f_idx].squeeze()
    ICD_vec = ICD_vec.abs()
    unrelated_ICD = ICD_vec < 0.01 # close to 0 
    unrelated_ICD = unrelated_ICD.nonzero().squeeze()
    icd9cm = InnerMap.load("ICD9CM")
    icd9proc = InnerMap.load("ICD9PROC")
    for i in range(n_random):
        n_codes = 0
        random_chunk = []
        random_desc = []
        while n_codes < k:
            random_ICD = random.choice(unrelated_ICD.tolist())
            ICD_code = label_transform.index2target[random_ICD]
            if ICD_code not in ICD_codes:
                if ICD_code in icd9cm:
                    random_desc.append(icd9cm.lookup(ICD_code))
                    random_chunk.append(ICD_code)
                    n_codes += 1
                elif ICD_code in icd9proc:
                    random_desc.append(icd9proc.lookup(ICD_code))
                    random_chunk.append(ICD_code)
                    n_codes += 1
        random_chunks.append(random_chunk)
        random_chunks_desc.append(random_desc)
    return random_chunks, random_chunks_desc
# tldr;
def get_global_interp_human_eval_dict(global_mat, contexts_summary, contexts, label_transform, n=100, topk=5, n_random=3):
    experiment_dict = {} 
    random_f_idxs = torch.randperm(global_mat.shape[1])[:n]
    max_dictionary_features = global_mat.max(dim=0).values
    max_f_vals, max_f_idxs = torch.sort(max_dictionary_features, descending=True)
    max_f_idxs = max_f_idxs[:n]
    icd9cm = InnerMap.load("ICD9CM")
    icd9proc = InnerMap.load("ICD9PROC")
    for idx in max_f_idxs:
        if idx.item() in contexts:
            # print(context_str)
            context_str = get_contexts(contexts, idx.item(), 4)
            if str(idx.item()) in contexts_summary and len(context_str) > 3:
                
                context_strings_formatted = ""
                for str_list in context_str:
                    context_strings_formatted += " ".join(str_list) + "\n"
                print("Feature:", idx)
                top_ICD_codes = extract_ICD_codes_per_feature(global_mat, idx, label_transform, topk)
                f_summary = contexts_summary[str(idx.item())]
                ICD_descriptions = []
                print(f_summary)
                print(context_strings_formatted)
                for icd in top_ICD_codes["top"]:
                    icd_desc = ""
                    if icd in icd9cm:
                        icd_desc = icd9cm.lookup(icd)
                    elif icd in icd9proc:
                        icd_desc = icd9proc.lookup(icd)
                    ICD_descriptions.append(icd_desc)
                    print(icd_desc)

                # unrelated_ICD_code_chunks, unrelated_ICD_code_descs = get_random_unrelated_ICD_codes(idx, global_mat, label_transform, top_ICD_codes["top_idx"], n_random, topk)
                # print(unrelated_ICD_code_descs)
                print("\n\n")
                experiment_dict[idx.item()] = {
                    "feature_summary": f_summary,
                    "context": context_strings_formatted,
                    "top_ICD_descs": ICD_descriptions,
                    "top_ICDs": top_ICD_codes["top"],
                }
        
    # sample other ICD codes from other dictionary features to create the "random set".
    f_ids = list(experiment_dict.keys())
    for f_id, val in experiment_dict.items():
        top_ICDs = experiment_dict[f_id]["top_ICDs"]
        i = 0 
        random_ICD_descs = []
        random_ICDs = []
        while i < n_random:
            random_f_id = random.choice(f_ids)
            if random_f_id != f_id:
                random_ICD_desc = experiment_dict[random_f_id]["top_ICD_descs"]
                random_ICD = experiment_dict[random_f_id]["top_ICDs"]
                if len(set(random_ICD).intersection(set(top_ICDs))) == 0:
                    random_ICD_descs.append(random_ICD_desc)
                    random_ICDs.append(random_ICD)
                    i += 1

        experiment_dict[f_id]["random_ICDs"] = random_ICDs
        experiment_dict[f_id]["random_ICD_descs"] = random_ICD_descs

    return experiment_dict


# get_top_dictionary_features(global_mat, L1_summaries, L1_reduced_contexts, 200, 5)
# get_random_dictionary_features(global_mat, L1_summaries, L1_reduced_contexts, 200, 5)
l1_experiment_dict = get_global_interp_human_eval_dict(global_mat, L1_summaries, L1_reduced_contexts, label_transform, n=100, topk=5, n_random=1)
print(len(l1_experiment_dict.keys()))


In [ ]:
def random_prettyprint_dict(input_dict, num_items=5):
    """
    Randomly selects and pretty prints 5 items from the given dictionary.

    Args:
        input_dict (dict): The dictionary to select items from.
        num_items (int, optional): The number of items to select and print. Defaults to 5.
    """
    # Get a list of the dictionary keys
    keys = list(input_dict.keys())

    # Randomly select 5 keys
    selected_keys = random.sample(keys, min(num_items, len(keys)))

    # Create a new dictionary with the selected keys
    selected_items = {k: input_dict[k] for k in selected_keys}

    # Pretty print the selected items
    pprint.pprint(selected_items)
random_prettyprint_dict(l1_experiment_dict, num_items=30)

In [ ]:
import matplotlib.pyplot as plt 
print(global_mat.shape)
plt.figure()
s = 400
img = plt.imshow(global_mat[:s, :s], cmap="hot")
# Add the colorbar
cbar = plt.colorbar(img)
# Set the colorbar label to show the minimum and maximum values
plt.xlabel("f")
plt.ylabel("ICD")
cbar.set_label(f"Magnitude (min={global_mat[:s, :s].min():.2f}, max={global_mat[:s, :s].max():.2f})")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

print(global_mat.shape)

plt.figure()

s = 50

img = plt.imshow(global_mat[:s, :s], cmap="hot")

# Remove the axis and ticks
plt.axis('off')

# Add the colorbar
cbar = plt.colorbar(img)

# Set the colorbar label to show the minimum and maximum values
# cbar.set_label(f"Magnitude (min={global_mat[:s, :s].min():.2f}, max={global_mat[:s, :s].max():.2f})")
cbar.set_label("Magnitude")
plt.savefig("figs/dict_attn/LDA.png")
plt.show()


# RELU

In [ ]:
plm_icd = load_plm_icd(plm_path, dataset, n_classes, device, use_finetuned_model)

In [ ]:
s = 100
relu_mat = plm_icd.attention.second_linear.weight.data
print(relu_mat.shape)
plt.figure()
img = plt.imshow(relu_mat[:s, :s], cmap="hot")
# Add the colorbar
cbar = plt.colorbar(img)
# Set the colorbar label to show the minimum and maximum values
# cbar.set_label(f"Magnitude (min={relu_mat[:s, :s].min():.2f}, max={relu_mat[:s, :s].max():.2f})")
cbar.set_label("Magnitude")
plt.xlabel("Z")
plt.ylabel("ICD")
plt.show()


In [ ]:
s = 50
plt.figure()
relu_mat = plm_icd.attention.second_linear.weight.data
img = plt.imshow(relu_mat[:s, :s], cmap="hot")
# Remove the axis and ticks
plt.axis('off')
# Add the colorbar
cbar = plt.colorbar(img)
# Set the colorbar label to show the minimum and maximum values
# cbar.set_label(f"Magnitude (min={relu_mat[:s, :s].min():.2f}, max={relu_mat[:s, :s].max():.2f})")
cbar.set_label("Magnitude")
plt.savefig("figs/dict_attn/laat.png")
plt.show()

In [ ]:
def sparsity_ratio(tensor):
    """
    Calculates the sparsity ratio of a PyTorch tensor.
    
    Args:
        tensor (torch.Tensor): The input tensor.
    
    Returns:
        float: The sparsity ratio of the tensor.
    """
    num_nonzero = torch.count_nonzero(tensor)
    return 1 - (num_nonzero / tensor.numel())

def sparse_density(tensor):
    """
    Calculates the sparse density of a PyTorch tensor.
    
    Args:
        tensor (torch.Tensor): The input tensor.
    
    Returns:
        float: The sparse density of the tensor.
    """
    num_nonzero = torch.count_nonzero(tensor)
    return num_nonzero / tensor.numel()

def percent_zeros(tensor):
    """
    Calculates the percentage of zeros in a PyTorch tensor.
    
    Args:
        tensor (torch.Tensor): The input tensor.
    
    Returns:
        float: The percentage of zeros in the tensor.
    """
    num_zeros = tensor.numel() - torch.count_nonzero(tensor)
    return (num_zeros / tensor.numel()) * 100

def effective_rank(tensor):
    """
    Calculates the effective rank of a PyTorch tensor.
    
    Args:
        tensor (torch.Tensor): The input tensor.
    
    Returns:
        int: The effective rank of the tensor.
    """
    return torch.linalg.matrix_rank(tensor).item()

def print_statistics_matrix(tensor):
    print("max:", tensor.max().item())
    print("min:", tensor.min().item())
    print("mean:", tensor.mean().item())
    print("std:", tensor.std().item())
    print(f"Sparsity ratio: {sparsity_ratio(tensor):.2f}")
    print(f"Sparse density: {sparse_density(tensor):.2f}")
    print(f"Percentage of zeros: {percent_zeros(tensor):.2f}%")
    print(f"Effective rank: {effective_rank(tensor)}")
    print("mean max per dict f:", tensor.max(1).values.mean().item())
    print("mean max per icd:", tensor.max(0).values.mean().item())

In [ ]:
# print_statistics_matrix((global_mat > 0.1).float())
# print_statistics_matrix(relu_mat)

In [ ]:
print(global_mat.shape)
print(plm_icd.attention.second_linear.weight.data.shape)
relu_mat = plm_icd.attention.second_linear.weight.data

In [ ]:
# random_prettyprint_dict(experiment_dict, num_items=30)

# merge all ICD codes into string separated by \n
def convert_global_dict_to_df(dict):
    df = {"fid": [], "truth": [],"contexts":[], "summary" : [], "choice1":[], "choice2":[]}
    for fid, items in dict.items():
        df["fid"].append(fid)
        top_icds = "\n".join(items["top_ICD_descs"])
        random_icds = "\n".join(items["random_ICD_descs"][0])
        choices = [top_icds, random_icds]
        contexts = "".join(items["context"])
        random.shuffle(choices)
        df["truth"].append(top_icds)
        df["contexts"].append(contexts)
        df["summary"].append(items["feature_summary"])
        df["choice1"].append(choices[0])
        df["choice2"].append(choices[1])
    return pd.DataFrame(df)
l1_experiment_df = convert_global_dict_to_df(l1_experiment_dict)
l1_experiment_df.head(20)

In [ ]:
relu_experiment_df = convert_global_dict_to_df(relu_experiment_dict)
relu_experiment_df.head(20)

In [ ]:
def save_global_questions_df(df, baseline, n=100):
    path = f"experiments/dict_att/mimiciii_clean/dictionary/questions/global/human_{baseline}_global_interp_questions_top{n}.csv"
    df.to_csv(path, index=False)

def load_global_questions_df(baseline, n=100):
    path = f"experiments/dict_att/mimiciii_clean/dictionary/questions/global/human_{baseline}_global_interp_questions_top{n}.csv"
    return pd.read_csv(path)

# get_top_relu_features(relu_mat, RELU_summaries, RELU_reduced,n=20, topk=4)
n = None
print(len(l1_experiment_dict))

l1_experiment_dict = get_global_interp_human_eval_dict(global_mat, L1_summaries, L1_reduced_contexts, label_transform, n=n, topk=5, n_random=1)
relu_experiment_dict = get_global_interp_human_eval_dict(relu_mat, RELU_summaries, RELU_reduced, label_transform, n=n, topk=5, n_random=1)
print(len(relu_experiment_dict.keys()))
l1_experiment_df = convert_global_dict_to_df(l1_experiment_dict)
relu_experiment_df = convert_global_dict_to_df(relu_experiment_dict)
 # so it errors out here
# l1_experiment_df = l1_experiment_df.sample(n=100, random_state=42, replace=False)
# relu_experiment_df = relu_experiment_df.sample(n=100, random_state=42, replace=False)
print(len(l1_experiment_df))
print(len(relu_experiment_df))
save_global_questions_df(l1_experiment_df, "L1", n="all")
save_global_questions_df(relu_experiment_df, "RELU",n="all")

In [ ]:
import einops 

# ablation experiments tokens vs. weight ablation
# def get_ablation_results(dataloader, model : SparsePLMICDClassifierModule, device, n_batches=10, quantile= 0.95, use_precise=True):
#     i = 0
#     sum = 0
#     iterator = iter(dataloader)
#     if n_batches == None:
#         n_batches = len(dataloader) # use all if None

#     # running sum 
#     n = 0 
    
#     p_delta_laat_not_targets = 0 
#     p_delta_dictionary_not_targets = 0
#     p_delta_top_cls_drop_laat = 0
#     p_delta_top_cls_drop_dict = 0
#     total_notes = 0
#     while i < n_batches:  
#         batch = next(iterator)
#         tokens, targets, attn_mask = batch.data, batch.targets, batch.attention_mask
#         b,c,t = tokens.shape
#         for note in range(b):
#             note_tokens = tokens[note].unsqueeze(0) # keep the batch size
#             note_attn_mask = attn_mask[note].unsqueeze(0)
#             # fwd pass, only get ground truth classes
#             og_pred = None
#             og_hidden_out = model.get_hidden_output(note_tokens.to(device), note_attn_mask.to(device))
#             og_hidden_out = og_hidden_out.view(1,c*t,-1)
#             with torch.no_grad():
#                 og_pred = torch.sigmoid(model.get_reconstructed_logits(og_hidden_out))
#             og_top_softmax, top_cls_idx = og_pred.max(dim=1) # get back the top class for each example, will do top k later.
            
#             #create reverse one-hot vector for none-top class
#             not_note_targets = torch.ones(og_pred.shape)
#             not_note_targets[:,top_cls_idx] = 0 # set the top class to 0, to ignore

#             # get attn
#             label_attn = model.classifier_head.attn.detach()
#             # top_cls_attn= top_cls.unsqueeze(1).unsqueeze(2).expand(-1,-1, label_attn.shape[2])
            
#             token_attn_top_cls = label_attn.squeeze()[top_cls_idx]
#             # print("top cls attn shape")
#             # print(token_attn_top_cls.shape)
#             threshold = torch.quantile(token_attn_top_cls, quantile, dim=1)
#             # print("threshold:", threshold.shape)
#             not_relevant_tokens = token_attn_top_cls < threshold
#             token_ablated_h_out = not_relevant_tokens.view(1,c*t,1) * og_hidden_out # keep only not_relevant_tokens
            
#             p_ablated = torch.sigmoid(model.get_reconstructed_logits(token_ablated_h_out))
#             p_delta_top_cls_drop_laat += (og_pred - p_ablated)[:,top_cls_idx].item()
#             delta_p_not_targets = not_note_targets.to(p_ablated.device) * (og_pred - p_ablated).abs()
#             p_delta_laat_not_targets += delta_p_not_targets.sum().item()
#             # print(delta_p_not_targets.sum().item())
       
        
#             # get the most relevant tokens and get their f acts
#             relevant_tokens = token_attn_top_cls >= threshold
#             relevant_tokens = torch.nonzero(relevant_tokens.view(-1))
       
      
#             f_acts, x_recon, aenc_loss_dict = model.dictionary(og_hidden_out.squeeze()) #
        
#             # print("f_acts", f_acts.shape)
#             f_acts = f_acts.squeeze()            
#             # get the f_id that corresponds to the top cls, specifically the f_ids that are activated for those specific tokens
#             most_relevant_f_acts = f_acts[relevant_tokens.squeeze()].squeeze() # should be relevant tokens x f
#             # print("most_relevant_f_acts:", most_relevant_f_acts.shape)
#             most_relevant_fids = torch.nonzero(most_relevant_f_acts.max(dim=0).values).squeeze() # none-zero are the most relevant f_ids
#             f_ablated_probs = torch.sigmoid(model.get_ablated_f_probs(og_hidden_out, most_relevant_fids, top_cls_idx, use_precise=use_precise)) # use precise first then, we run not use_precise
#             p_most_relevant_to_cls = og_pred - f_ablated_probs
#             p_delta_top_cls_drop_dict += p_most_relevant_to_cls[0,top_cls_idx].item()
#             deltas_not_targets_f_ablated = not_note_targets.to(p_most_relevant_to_cls.device) * p_most_relevant_to_cls.abs()
#             p_delta_dictionary_not_targets += deltas_not_targets_f_ablated.sum().item()
            
#             total_notes+=1
#             print("Total Notes:", total_notes)
#             # print(p_delta_dictionary_not_targets)
#             # print(deltas_not_targets_f_ablated.sum().item())
        
#         i+=1 
#         # print(i)
#     print("p_delta_laat_top-cls", p_delta_top_cls_drop_laat / total_notes)
#     print("p_delta_dict_top-cls", p_delta_top_cls_drop_dict / total_notes)
#     p_delta_laat_not_targets = p_delta_laat_not_targets / total_notes
#     p_delta_dictionary_not_targets = p_delta_dictionary_not_targets / total_notes
#     print("p_delta_laat_not_targets", p_delta_laat_not_targets)
#     print("p_delta_dictionary_not_targets", p_delta_dictionary_not_targets)
#     p_delta_top_cls_drop_dict = p_delta_top_cls_drop_dict / total_notes
#     p_delta_top_cls_drop_laat = p_delta_top_cls_drop_laat / total_notes
#     return p_delta_laat_not_targets, p_delta_dictionary_not_targets, p_delta_top_cls_drop_laat, p_delta_top_cls_drop_dict

In [ ]:
# model = load_dma_experiment(run_id, dataset, dictionary_type, device)
# import os
# os.environ['TORCH_USE_CUDA_DSA'] = '1'
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# gpu = 7
# device = torch.device(f"cuda:{gpu}")
# device = torch.device("cpu")
# model = load_dma_experiment(run_id, dataset, dictionary_type, device)
model.to(device)
get_ablation_results(test_loader, model, device, n_batches=None, quantile=0.95, use_precise=False)


In [ ]:
get_ablation_results(test_loader, model, device, n_batches=None, quantile=0.95, use_precise=True)

In [ ]:
import torch

def get_ablation_results(dataloader, model: SparsePLMICDClassifierModule, device, n_batches=10, quantile=0.95, use_precise=True, baseline="random"):
    iterator = iter(dataloader)
    if n_batches is None:
        n_batches = len(dataloader)  # use all if None

    total_notes = 0
    metrics = {
        'p_delta_laat_not_targets': [],
        'p_delta_dictionary_not_targets': [],
        'p_delta_top_cls_drop_laat': [],
        'p_delta_top_cls_drop_dict': []
    }

    for i in range(n_batches):
        batch = next(iterator)
        tokens, targets, attn_mask = batch.data, batch.targets, batch.attention_mask
        b, c, t = tokens.shape

        for note in range(b):
            note_tokens = tokens[note].unsqueeze(0)  # keep the batch size
            note_attn_mask = attn_mask[note].unsqueeze(0)

            # fwd pass, only get ground truth classes
            og_hidden_out = model.get_hidden_output(note_tokens.to(device), note_attn_mask.to(device))
            og_hidden_out = og_hidden_out.view(1, c * t, -1)
            with torch.no_grad():
                og_pred = torch.sigmoid(model.get_reconstructed_logits(og_hidden_out))
            og_top_softmax, top_cls_idx = og_pred.max(dim=1)  # get back the top class for each example

            # create reverse one-hot vector for none-top class
            not_note_targets = torch.ones(og_pred.shape)
            not_note_targets[:, top_cls_idx] = 0  # set the top class to 0, to ignore

            # get attn
            label_attn = model.classifier_head.attn.detach()
            token_attn_top_cls = label_attn.squeeze()[top_cls_idx]
            threshold = torch.quantile(token_attn_top_cls, quantile, dim=1)
            not_relevant_tokens = token_attn_top_cls < threshold
            relevant_tokens = token_attn_top_cls >= threshold
            p_ablated = None
            if baseline != "replace":
                token_ablated_h_out = None
                if baseline == "random":
                    token_ablated_h_out = og_hidden_out + relevant_tokens.view(1,-1,1) * 2 * torch.randn_like(og_hidden_out)
                elif baseline == "ablate":
                    token_ablated_h_out = not_relevant_tokens.view(1, c * t, 1) * og_hidden_out  # keep only not_relevant_tokens
                p_ablated = torch.sigmoid(model.get_reconstructed_logits(token_ablated_h_out))
            else:
                # set the tokens to id 0 to ablate instead of the embeddings
                # print(not_relevant_tokens.shape)
                # print(note_tokens.shape)
                replaced_tokens = not_relevant_tokens.view(1, c, t) * note_tokens.to(device) # keep the attention mask so the model knows about it!
                p_ablated, _ = model(replaced_tokens.to(device), note_attn_mask.to(device))
                p_ablated = torch.sigmoid(p_ablated)

            p_delta_top_cls_drop_laat = (og_pred - p_ablated)[:, top_cls_idx].item()
            delta_p_not_targets = not_note_targets.to(p_ablated.device) * (og_pred - p_ablated).abs()
            p_delta_laat_not_targets = delta_p_not_targets.sum().item()

            relevant_tokens = token_attn_top_cls >= threshold
            relevant_tokens = torch.nonzero(relevant_tokens.view(-1))

            f_acts, x_recon, aenc_loss_dict = model.dictionary(og_hidden_out.squeeze())
            f_acts = f_acts.squeeze()
            most_relevant_f_acts = f_acts[relevant_tokens.squeeze()].squeeze()  # should be relevant tokens x f
            most_relevant_fids = torch.nonzero(most_relevant_f_acts.max(dim=0).values).squeeze()  # non-zero are the most relevant f_ids
            f_ablated_probs = torch.sigmoid(model.get_ablated_f_probs(og_hidden_out, most_relevant_fids, top_cls_idx, use_precise=use_precise))
            p_most_relevant_to_cls = og_pred - f_ablated_probs
            p_delta_top_cls_drop_dict = p_most_relevant_to_cls[0, top_cls_idx].item()
            deltas_not_targets_f_ablated = not_note_targets.to(p_most_relevant_to_cls.device) * p_most_relevant_to_cls.abs()
            p_delta_dictionary_not_targets = deltas_not_targets_f_ablated.sum().item()

            total_notes += 1
            print(f"Total Notes: {total_notes}")

            metrics['p_delta_laat_not_targets'].append(p_delta_laat_not_targets)
            metrics['p_delta_dictionary_not_targets'].append(p_delta_dictionary_not_targets)
            metrics['p_delta_top_cls_drop_laat'].append(p_delta_top_cls_drop_laat)
            metrics['p_delta_top_cls_drop_dict'].append(p_delta_top_cls_drop_dict)

    # Compute means and standard deviations
    for metric, values in metrics.items():
        mean = sum(values) / total_notes
        variance = sum((x - mean) ** 2 for x in values) / (total_notes - 1)
        std_dev = torch.sqrt(torch.tensor(variance)).item()
        metrics[metric] = {'mean': mean, 'std_dev': std_dev}

    print(f"Metrics: {metrics}")
    return metrics


gpu = 0
device = torch.device(f"cuda:{gpu}")
model.to(device)
get_ablation_results(test_loader, model, device, n_batches=None, quantile=0.95, use_precise=True, baseline="random")

In [ ]:
get_ablation_results(test_loader, model, device, n_batches=None, quantile=0.95, use_precise=True, baseline="replace")

In [ ]:
import shap
import numpy as np
from sklearn.linear_model import LinearRegression

iterator = iter(test_loader)
batch = next(iterator)
tokens, targets, attn_mask = batch.data, batch.targets, batch.attention_mask
plm_embeddings = model.get_hidden_output(tokens.to(device), attn_mask.to(device))
# let's just convert it to 
print(plm_embeddings.shape)

In [ ]:
import shap
import numpy as np
import torch
import torch.nn as nn

# Define the toy model
class ToyModel(nn.Module):
    def __init__(self, model : SparsePLMICDClassifierModule):
        super(ToyModel, self).__init__()
        self.model = model

    # assumes x is a 2D tensor of B x C*768, so we need to reshape it back into non-2D tensor, of B,C, 768
    def forward(self, x):
        b, d = x.shape
        x = x.view(b,-1,768) # something like this
        p = self.model.get_reconstructed_logits(x)
        return p

# Create and train the toy model
gpu = 0
device = torch.device(f"cuda:{gpu}")
model.to(device)
b, s, nc,cs = plm_embeddings.shape
X_tensor = plm_embeddings.view(b, s*nc*cs).to(device)
toy_model = ToyModel(model)
toy_model.to(device)
# train_toy_model(toy_model, X_tensor, y_tensor)

# Create the GradientExplainer
explainer = shap.GradientExplainer(toy_model, X_tensor)

# Get the Shapley values for a single instance
instance = X_tensor[0:1]  # Select the first instance
shap_values = explainer.shap_values(instance)

# Print the Shapley values
print("Shapley values:")
print(shap_values)

In [ ]:
import time

# Start the timer
start_time = time.time()

# label attention computation, simply just forward operation
note_tokens = tokens[0].unsqueeze(0)  # keep the batch size
note_attn_mask = attn_mask[0].unsqueeze(0)
b, c, t = note_tokens.shape

# fwd pass, only get ground truth classes
og_hidden_out = model.get_hidden_output(note_tokens.to(device), note_attn_mask.to(device))
og_hidden_out = og_hidden_out.view(1, c * t, -1)

with torch.no_grad():
    og_pred = torch.sigmoid(model.get_reconstructed_logits(og_hidden_out))
    og_top_softmax, top_cls_idx = og_pred.max(dim=1)  # get back the top class for each example

# get attn
label_attn = model.classifier_head.attn.detach()

# End the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time in seconds
print(f"Elapsed time: {elapsed_time:.5f} seconds")

In [ ]:
import time

# Start the timer
start_time = time.time()

# label attention computation, simply just forward operation
note_tokens = tokens[0].unsqueeze(0)  # keep the batch size
note_attn_mask = attn_mask[0].unsqueeze(0)
b, c, t = note_tokens.shape

# fwd pass, only get ground truth classes
og_hidden_out = model.get_hidden_output(note_tokens.to(device), note_attn_mask.to(device))
og_hidden_out = og_hidden_out.view(1, c * t, -1)

model.classifier_head.dictionary.encode(og_hidden_out.squeeze()).shape

# og_top_softmax, top_cls_idx = og_pred.max(dim=1) # get back the top class for each example

# get attn
# label_attn = model.classifier_head.attn.detach()

# End the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time in seconds
print(f"Elapsed time: {elapsed_time:.5f} seconds")

In [ ]:
import time

# Start the timer
start_time = time.time()

model.classifier_head.icd_memory.data.shape
end_time = time.time()
elapsed_time = end_time - start_time
# Print the elapsed time in seconds
print(f"Elapsed time: {elapsed_time:.5f} seconds")

In [ ]:
get_ablation_results(test_loader, model, device, n_batches=None, quantile=0.95, use_precise=False)

In [ ]:
# global debugging case study?

def get_top_classes(dataloader, n_batches=None, n_classes=3681):
    iterator = iter(dataloader)
    total_targets = torch.zeros(n_classes)
    if n_batches is None:
        n_batches = len(dataloader)  # use all if None
    for i in range(n_batches):
        batch = next(iterator)
        tokens, targets, attn_mask = batch.data, batch.targets, batch.attention_mask
        total_targets += targets.sum(dim=0)
    return total_targets

total_per_cls = get_top_classes(test_loader)

In [ ]:
total_num, cls_idxs = total_per_cls.sort(descending=True)
# label_transform.index2target[862]
top_icd = label_transform.index2target[cls_idxs[0].item()]
rarest_icd = label_transform.index2target[cls_idxs[-1].item()]
print(top_icd)
print("n:", total_num[0])
print(rarest_icd)
print("n:", total_num[-1])

# Undercoverage Case (Actually Predicted lol)

In [ ]:
import nltk 
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')  # For tokenization
nltk.download('stopwords')  # For stop words
def no_whitespace(word):
    return re.sub(r"\s+", "", word)
icd9cm = InnerMap.load("ICD9CM")
icd9proc = InnerMap.load("ICD9PROC")
if rarest_icd in icd9cm:
    rare_icd_desc = icd9cm.lookup(rarest_icd)
else:
    rare_icd_desc = icd9proc.lookup(rarest_icd)
top_icd_desc = icd9cm.lookup(top_icd)
print(rare_icd_desc)
rare_tokens = text_transform.transform(rare_icd_desc)
print(rare_tokens)
stop_words = set(stopwords.words('english'))
filtered_desc_tokens = [no_whitespace(word.lower()) for word in word_tokenize(rare_icd_desc) if not word in stop_words]
# we need to tokenize the description
# filtered_desc_tokens = [token for token in filtered_desc_tokens if re.search(r'[a-zA-Z]', token)]
# filtered_desc_tokens = [text_transform.tokenizer.encode(token)[1:-1] for token in filtered_desc_tokens] 
# filtered_desc_tokens = [item for sublist in filtered_desc_tokens for item in sublist]
# print(filtered_desc_tokens)

In [ ]:
print(rare_tokens["input_ids"][0])
print(filtered_desc_tokens)

In [ ]:
def get_text_f_ids(desc, text_transform, model : SparsePLMICDClassifierModule, device):
    transformed = text_transform.transform(rare_icd_desc)
    attn_mask = transformed["attention_mask"][0][1:-1]
    token_ids = transformed["input_ids"][0][1:-1]
    h_out = model.get_hidden_output(token_ids.view(1,1,-1).to(device), attn_mask.view(1,1,-1).to(device))
    features = model.classifier_head.dictionary.encode(h_out).squeeze().squeeze()
    f_activated_desc = features.max(dim=0).values
    f_val_desc, f_ids_desc = f_activated_desc.sort(descending=True)
    return  f_val_desc, f_ids_desc

In [ ]:
text_transform.inverse_transform(rare_tokens["input_ids"][0][1:-1])

In [ ]:
rare_attn_mask = rare_tokens["attention_mask"][0][1:-1]
rare_token_ids = rare_tokens["input_ids"][0][1:-1]
h_out = model.get_hidden_output(rare_token_ids.view(1,1,-1).to(device), rare_attn_mask.view(1,1,-1).to(device))

In [ ]:
features = model.classifier_head.dictionary.encode(h_out).squeeze().squeeze()
f_activated_desc = features.max(dim=0).values
f_val_desc, f_ids_desc = f_activated_desc.sort(descending=True)
print(f_val_desc)
print((f_val_desc > 3).sum())
print(f_ids_desc[:8])

In [ ]:
def get_batches_with_cls(dataloader, cls_idx, n_batches=None):
    iterator = iter(dataloader)
    sample_ids = []
    sample_attn_masks = []
    sample_targets = []

    if n_batches is None:
        n_batches = len(dataloader)  # use all if None

    for i in range(n_batches):
        batch = next(iterator)
        tokens, targets, attn_mask = batch.data, batch.targets, batch.attention_mask
        for b in range(tokens.shape[0]):
            if targets[b, cls_idx] == 1:
                sample_ids.append(tokens[b])
                sample_attn_masks.append(attn_mask[b])
                sample_targets.append(targets[b])

    if len(sample_ids) == 0:
        # No samples with the class found
        return torch.empty(0), torch.empty(0), torch.empty(0)
    elif len(sample_ids) == 1:
        # Only one sample with the class found
        return sample_ids[0].unsqueeze(0), sample_attn_masks[0].unsqueeze(0), sample_targets[0].unsqueeze(0)
    else:
        # Multiple samples with the class found
        return sample_ids, sample_attn_masks, sample_targets
# sample_ids, sample_attn_masks, sample_targets = get_batches_with_cls(test_loader, cls_idxs[-1].item())
# print(sample_ids.shape)

In [ ]:
import einops
def get_sample_f_ids(sample_ids, sample_attn_masks, model, device):
    h_sample = model.get_hidden_output(sample_ids.to(device), sample_attn_masks.to(device)).squeeze()
    f_sample = model.classifier_head.dictionary.encode(einops.rearrange(h_sample, "b s d -> (b s) d")).squeeze().squeeze()
    f_sample_activated = f_sample.max(dim=0).values
    f_val_sample, f_ids_sample = f_sample_activated.sort(descending=True)
    return f_val_sample.view(-1), f_ids_sample.view(-1)
# h_sample = model.get_hidden_output(sample_ids.to(device), sample_attn_masks.to(device)).squeeze()
# print(h_sample.shape)
# f_sample = model.classifier_head.dictionary.encode(einops.rearrange(h_sample, "b s d -> (b s) d")).squeeze().squeeze()
# print(f_sample.shape)
# f_sample_activated = f_sample.max(dim=0).values
# f_val_sample, f_ids_sample = f_sample_activated.sort(descending=True)
# print(f_val_sample.shape)
# print((f_val_sample > 3).sum())
# print(f_ids_sample.shape)

In [ ]:
def get_overlapping_fids(tensor1, tensor2):
    # Convert the tensors to sets
    set1 = set(tensor1.view(-1).tolist())
    set2 = set(tensor2.view(-1).tolist())

    # Find the intersection of the sets
    overlapping_fids = set1.intersection(set2)
    # Convert the set of overlapping f_ids to a torch tensor
    overlapping_fids_tensor = torch.tensor(list(overlapping_fids), dtype=torch.int64)

    return overlapping_fids_tensor
overlap = get_overlapping_fids(f_ids_sample.view(-1)[:10], f_ids_desc.view(-1)[:10])
print(overlap)
print(len(overlap))
print(global_mat.shape)
print(global_mat[cls_idxs[-1], overlap])
for  fid in overlap:
    # print(L1_summaries[str(fid.item())])
    print(L1_reduced_contexts[fid.item()]["context_string"])

In [ ]:
token_overlap = get_overlapping_fids(rare_token_ids, sample_ids.view(-1))

In [ ]:
text_transform.inverse_transform(token_overlap)

In [ ]:
thumb_reattachment_sentence1 = "The surgeon successfully performed a thumb reattachment after the patient's traumatic accident."
thumb_reattachment_sentence2 = "Advances in microsurgery have made thumb reattachment a viable option for many individuals who have suffered a severe injury."
thumb_reattachment_sentence3 = "The intricate process of thumb reattachment involves reconnecting blood vessels, nerves, and tendons to restore function and sensation."
thumb_reattachment_sentence4 = "After the thumb reattachment surgery, the patient underwent extensive physical therapy to regain strength and dexterity in the affected hand."

In [ ]:
# does the model correctly predict the thumb reattachment code?
pred, aenc_loss = model(sample_ids, sample_attn_masks)
pred = torch.sigmoid(pred)
print((pred)[0,cls_idxs[-1]])


In [ ]:
print(pred.shape)

# Wrongly Predicted Cases!

In [ ]:
from collections import defaultdict

def get_most_wrongly_predicted_cls(dataloader, model, device, n_batches=None):
    model.eval()
    wrong_predictions = defaultdict(int)
    
    iterator = iter(dataloader)
    
    if n_batches is None:
        n_batches = len(dataloader)  # use all if None
    
    with torch.no_grad():
        for i in range(n_batches):
            batch = next(iterator)
            tokens, targets, attn_mask = batch.data, batch.targets, batch.attention_mask
            
            outputs, aenc_dict = model(tokens.to(device), attn_mask.to(device))
            predicted = (torch.sigmoid(outputs) > 0.3).cpu()
            # print(predicted.shape)
            for cls_idx in range(targets.shape[1]):
                cls_targets = targets[:, cls_idx]
                cls_predicted = predicted[:, cls_idx]
                
                wrong_cls_predictions = (cls_targets != cls_predicted).sum().item()
                wrong_predictions[cls_idx] += wrong_cls_predictions
    
    sorted_wrong_predictions = sorted(wrong_predictions.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_wrong_predictions

def get_most_wrongly_predicted_cls_false_positives(dataloader, model, device, n_batches=None):
    model.eval()
    false_positive_predictions = defaultdict(int)

    iterator = iter(dataloader)

    if n_batches is None:
        n_batches = len(dataloader)  # use all if None

    with torch.no_grad():
        for i in range(n_batches):
            batch = next(iterator)
            tokens, targets, attn_mask = batch.data, batch.targets, batch.attention_mask

            outputs, aenc_dict = model(tokens.to(device), attn_mask.to(device))
            predicted = (torch.sigmoid(outputs) > 0.3).cpu()

            for cls_idx in range(targets.shape[1]):
                cls_targets = targets[:, cls_idx]
                cls_predicted = predicted[:, cls_idx]

                # Identify false positives
                false_positives = (cls_targets == 0) & (cls_predicted == 1)
                false_positive_count = false_positives.sum().item()

                false_positive_predictions[cls_idx] += false_positive_count

    sorted_false_positive_predictions = sorted(false_positive_predictions.items(), key=lambda x: x[1], reverse=True)
    return sorted_false_positive_predictions
gpu = 3
device = torch.device(f"cuda:{gpu}")
print(model.device)
model.to(device)
# device = torch.device("cpu")

false_positive_predictions = get_most_wrongly_predicted_cls_false_positives(test_loader, model, device, n_batches=None)

In [ ]:
false_positive_predictions

In [ ]:
wrong_predictions = get_most_wrongly_predicted_cls(test_loader, model, device, n_batches=None)#
# wrong_predictions
# false_positive_predictions
# sanity = iter(test_loader)
# batch = next(sanity)
# targets = batch.targets
# print(targets.shape)

In [ ]:
fp_dict = {}
wrong_dict = {}
false_negative_counts = {}
for cls_idx, count in false_positive_predictions:
    fp_dict[cls_idx] = count
for cls_idx, count in wrong_predictions:
    wrong_dict[cls_idx] = count 
for cls_idx, count in fp_dict.items():
    if wrong_dict[cls_idx] > 200:
        false_negative_counts[cls_idx] = (wrong_dict[cls_idx] - count) / wrong_dict[cls_idx] # normalize by count!

sorted_false_negative_predictions = sorted(false_negative_counts.items(), key=lambda x: x[1], reverse=False)
print(sorted_false_negative_predictions)
fp_cls_idx = 3213
fp_cls = label_transform.index2target[3213] # false positive example
icd9cm = InnerMap.load("ICD9CM")
icd9proc = InnerMap.load("ICD9PROC")
if fp_cls in icd9cm:
    fp_icd_desc = icd9cm.lookup(fp_cls)
else:
    fp_icd_desc = icd9proc.lookup(fp_cls)
print(fp_cls)
print(fp_icd_desc)
val, fp_fid = global_mat[fp_cls_idx,:].sort(descending=True)
i = 0
for fid in fp_fid[:40].tolist():
    i+=1
    if fid in L1_reduced_contexts:
        # print(L1_summaries[str(fid)])
        print(fid)
        context_strings = [" ".join(token) for token in L1_contexts[fid]["context_string"]]
        context_string = "\n".join(context_strings)
        print(context_string)
        print(val[i])
        print("")
# for cls_idx, count in wrong_predictions:
    # overall_wrong_classes.append(cls_idx)
# for cls_idx, count in false_positive_predictions:
#     if cls_idx not in overall_wrong_classes:
#         print(f"False Positives: Index: {cls_idx} Code: {label_transform.index2target[fid]}: {count}")

In [ ]:
most_wrongly_predicted_cls_idx = 1024

In [ ]:
print(most_wrongly_predicted_cls_idx)
icd9cm = InnerMap.load("ICD9CM")
icd9proc = InnerMap.load("ICD9PROC")
most_wrongly_predicted_cls = label_transform.index2target[most_wrongly_predicted_cls_idx]
print(most_wrongly_predicted_cls)
print(icd9proc.lookup(most_wrongly_predicted_cls))
# icd9cm.lookup(most_wrongly_predicted_cls)
learned_vals, learned_f_ids = global_mat[most_wrongly_predicted_cls_idx].sort(descending=True)

In [ ]:
# torch.multiprocessing.set_sharing_strategy('file_system')
def get_batches_with_cls(dataloader, cls_idx, n_batches=None):
    iterator = iter(dataloader)
    sample_ids = []
    sample_attn_masks = []
    sample_targets = []

    if n_batches is None:
        n_batches = len(dataloader)  # use all if None

    for i in range(n_batches):
        batch = next(iterator)
        tokens, targets, attn_mask = batch.data, batch.targets, batch.attention_mask
        for b in range(tokens.shape[0]):
            if targets[b, cls_idx] == 1:
                sample_ids.append(tokens[b])
                sample_attn_masks.append(attn_mask[b])
                sample_targets.append(targets[b])

    if len(sample_ids) == 0:
        # No samples with the class found
        return torch.empty(0), torch.empty(0), torch.empty(0)
    elif len(sample_ids) == 1:
        # Only one sample with the class found
        return sample_ids[0].unsqueeze(0), sample_attn_masks[0].unsqueeze(0), sample_targets[0].unsqueeze(0)
    else:
        # Multiple samples with the class found
        return sample_ids, sample_attn_masks, sample_targets

def get_batches_without_cls(dataloader, cls_idx, n_batches=None):
    iterator = iter(dataloader)
    sample_ids = []
    sample_attn_masks = []
    sample_targets = []

    if n_batches is None:
        n_batches = len(dataloader)  # use all if None

    for i in range(n_batches):
        batch = next(iterator)
        tokens, targets, attn_mask = batch.data, batch.targets, batch.attention_mask
        for b in range(tokens.shape[0]):
            if targets[b, cls_idx] == 0:
                sample_ids.append(tokens[b])
                sample_attn_masks.append(attn_mask[b])
                sample_targets.append(targets[b])

    if len(sample_ids) == 0:
        # No samples without the class found
        return torch.empty(0), torch.empty(0), torch.empty(0)
    elif len(sample_ids) == 1:
        # Only one sample without the class found
        return sample_ids[0].unsqueeze(0), sample_attn_masks[0].unsqueeze(0), sample_targets[0].unsqueeze(0)
    else:
        # Multiple samples without the class found
        return sample_ids, sample_attn_masks, sample_targets

sample_ids, sample_attn_mask, sample_targets = get_batches_with_cls(test_loader, 3213)

In [ ]:
def get_text_f_ids(desc, text_transform, model : SparsePLMICDClassifierModule, device):
    model.to(device)
    transformed = text_transform.transform(desc)
    attn_mask = transformed["attention_mask"][0][1:-1]
    token_ids = transformed["input_ids"][0][1:-1]
    with torch.no_grad():
        h_out = model.get_hidden_output(token_ids.view(1,1,-1).to(device), attn_mask.view(1,1,-1).to(device))
        features = model.classifier_head.dictionary.encode(h_out).squeeze().squeeze()
        f_activated_desc = features.max(dim=0).values
        f_val_desc, f_ids_desc = f_activated_desc.sort(descending=True)
    return  f_val_desc.cpu(), f_ids_desc.cpu()

f_val_desc, f_ids_desc = get_text_f_ids(icd9proc.lookup(most_wrongly_predicted_cls), text_transform, model, device)

In [ ]:
print(f_ids_desc[:10])
print(f_val_desc[:10])

In [ ]:
print(sample_ids[0].shape)

In [ ]:
sample_f_val, sample_f_ids = get_sample_f_ids(sample_ids[1].unsqueeze(0), sample_attn_mask[1].unsqueeze(0), model, device)

In [ ]:
def get_overlapping_fids(tensor1, tensor2):
    # Convert the tensors to sets
    set1 = set(tensor1.view(-1).tolist())
    set2 = set(tensor2.view(-1).tolist())

    # Find the intersection of the sets
    overlapping_fids = set1.intersection(set2)
    # Convert the set of overlapping f_ids to a torch tensor
    overlapping_fids_tensor = torch.tensor(list(overlapping_fids), dtype=torch.int64)

    return overlapping_fids_tensor
n = 8
print(get_overlapping_fids(f_ids_desc.view(-1)[:n], learned_f_ids.view(-1)[:n]))
print(learned_f_ids[:n])
print(learned_vals[:n])
print(f_ids_desc[:n])
print(f_val_desc[:n])

In [ ]:
overlapping_sample_fids = get_overlapping_fids(sample_f_ids.view(-1)[:100].detach(), learned_f_ids.view(-1)[:100])
for i in range(500):
    if sample_f_ids[i] in overlapping_sample_fids:
        # print(i)
        print(sample_f_ids[i])
        print(sample_f_val[i])
print(" MATRIX WEIGHTS")
for i in range(100):
    if learned_f_ids[i] in overlapping_sample_fids:
        # print(i)
        print(learned_f_ids[i])
        print(learned_vals[i])
        print(L1_contexts[learned_f_ids[i].item()]["context_string"])
    # print(L1_summaries[str(163)])

In [ ]:
overlapping_sample_fids_icd_desc = get_overlapping_fids(sample_f_ids.view(-1)[:100], f_ids_desc.view(-1)[:4])
for i in range(100):
    if sample_f_ids[i] in overlapping_sample_fids_icd_desc:
        print(sample_f_ids[i])
        print(sample_f_val[i])
print("DESC WEIGHTS")
for i in range(4):
    if f_ids_desc[i] in overlapping_sample_fids_icd_desc:
        print(f_ids_desc[i])
        print(f_val_desc[i])
        print(L1_reduced_contexts[f_ids_desc[i].item()]["context_string"])

In [ ]:
import einops
def check_sample_predict_cls(tokens, attn_mask, targets, model, device, cls_idx):
    outputs, aenc_dict = model(tokens.to(device), attn_mask.to(device))
    predicted = (torch.sigmoid(outputs) > 0.3).cpu()
    cls_targets = targets[:, cls_idx]
    cls_predicted = predicted[:, cls_idx]
    wrong_cls_predictions = (cls_targets != cls_predicted).squeeze()
    return wrong_cls_predictions

def get_sample_f_ids(sample_ids, sample_attn_masks, model, device):
    h_sample = model.get_hidden_output(sample_ids.to(device), sample_attn_masks.to(device)).squeeze()
    f_sample = model.classifier_head.dictionary.encode(einops.rearrange(h_sample, "b s d -> (b s) d")).squeeze().squeeze()
    f_sample_activated = f_sample.max(dim=0).values
    f_val_sample, f_ids_sample = f_sample_activated.sort(descending=True)
    return f_val_sample.view(-1), f_ids_sample.view(-1)

# check_sample_predict_cls(sample_ids[1].unsqueeze(0), sample_attn_mask[1].unsqueeze(0), sample_targets[1].unsqueeze(0), model, device, most_wrongly_predicted_cls_idx)



In [ ]:
# get the most common fids across each wrongly predicted sample
def identify_most_common_fids_in_wrongly_predicted_cls(sample_ids, sample_attn_masks, sample_targets, model, device, most_wrongly_predicted_cls_idx, topk=100):
    most_common_fids = {} # make a running count
    for i in range(len(sample_ids)):
        if not check_sample_predict_cls(sample_ids[i].unsqueeze(0), sample_attn_mask[i].unsqueeze(0), sample_targets[i].unsqueeze(0), model, device, most_wrongly_predicted_cls_idx):
            sample_f_val, sample_f_ids = get_sample_f_ids(sample_ids[i].unsqueeze(0), sample_attn_masks[i].unsqueeze(0), model, device)
            sample_f_ids = sample_f_ids.squeeze().cpu().detach()
            sample_f_ids = sample_f_ids[:topk]
            for f_id in sample_f_ids:
                if f_id.item() in most_common_fids:
                    most_common_fids[f_id.item()] += 1
                else:
                    most_common_fids[f_id.item()] = 1
        # print(most_common_fids)
    most_common_fids = sorted(most_common_fids.items(), key=lambda x: x[1], reverse=True)
    return most_common_fids

gpu = 0
device = torch.device(f"cuda:{gpu}")
model.to(device)
fids = identify_most_common_fids_in_wrongly_predicted_cls(sample_ids, sample_attn_mask, sample_targets, model, device, 3213)

In [ ]:
fn_count = {}
for i in range(100):
    fid = fids[i][0]
    count = fids[i][1]
    fn_count[fid] = count
    if fid in L1_reduced_contexts:
        # print(L1_summaries[str(fid)])
        print(fid)
        context_strings = [" ".join(token) for token in L1_contexts[fid]["context_string"]]
        context_string = "\n".join(context_strings)
        print(context_string)
        print("")
        # print(L1_contexts[fid]["context_string"])

In [ ]:
val, fp_fid = global_mat[3213,:].sort(descending=True)
overlap = get_overlapping_fids(fp_fid[:2000], torch.tensor(list(fn_count.keys())[:2000]))
i = 0
with open("output.txt", "w") as f:
    for fid in overlap.tolist():
        if fid in L1_reduced_contexts:
            associative_score = global_mat[3213, fid].item()
            feature_score = fn_count[fid]
            if associative_score < 2.0 and feature_score > 50:
                f.write(str(fid) + "\n")
                context_strings = [" ".join(token) for token in L1_contexts[fid]["context_string"]]
                context_string = "\n".join(context_strings)
                f.write(context_string + "\n")
                f.write("associative score: " + str(global_mat[fp_cls_idx, fid].item()) + "\n")
                f.write("\n")
            else:
                f.write("\n")
# 5387 - directly related fid not used.
direct_related_fid = 5387
semi_related_fids = [5297, 5836, 3311, 1775, 3320, 764, 4357, 4368, 1306, 4896, 3920, 1889, 907, 4500, 5545, 3015, 5066, 988]

In [ ]:
# print(fids)
# print(L1_contexts[fids[0][0]]["context_string"])
for fid, count in fids:
    if str(fid) in L1_summaries:
        print(fid)
        print(count)
        print(L1_summaries[str(fid)])
        # print(L1_contexts[fid]["context_string"])

In [ ]:
# manually upweight experiment of a single ICD code to see if it improves the prediction
potentially_causal_fids = []
for fid, count in fids:
    if str(fid) in L1_summaries:
        print("Fid:", fid)
        print("Count:", count)
        if str(fid) in L1_summaries:
            print(L1_summaries[str(fid)])
            # print(L1_contexts[fid]["context_string"])
            print(global_mat[most_wrongly_predicted_cls_idx, fid])
            if count > 1000 and fid != 638:
                potentially_causal_fids.append((fid,count))


In [ ]:
model.classifier_head.icd_memory.data.shape
potentially_causal_fids

# Debugging Case Study Experiment

In [ ]:
import importlib

from einops import rearrange
import train_lda_ms
import label_dict_att
import numpy as np 
importlib.reload(train_lda_ms)
importlib.reload(label_dict_att)
from train_lda_ms import SparsePLMICDClassifierModule
from pyhealth.metrics import multilabel_metrics_fn


def get_false_positive_samples(model, dataloader, cls_idx, n_batches=None, device='cpu'):
    model.eval()
    iterator = iter(dataloader)
    false_positive_ids = []
    false_positive_attn_masks = []
    false_positive_targets = []

    if n_batches is None:
        n_batches = len(dataloader)  # use all if None

    for i in range(n_batches):
        batch = next(iterator)
        tokens, targets, attn_mask = batch.data, batch.targets, batch.attention_mask

        # Move the batch to the specified device
        # tokens = tokens.to(device)
        # attn_mask = attn_mask.to(device)
        # targets = targets.to(device)

        with torch.no_grad():
            outputs, _ = model(tokens.to(device), attention_mask=attn_mask.to(device))
            predictions = (torch.sigmoid(outputs) > 0.3).cpu()
        predictions= predictions[:, cls_idx]
        for b in range(tokens.shape[0]):
            if predictions[b] == 1 and targets[b, cls_idx] == 0:
                false_positive_ids.append(tokens[b])
                false_positive_attn_masks.append(attn_mask[b])
                false_positive_targets.append(targets[b])

    if len(false_positive_ids) == 0:
        # No false positive samples found
        return torch.empty(0), torch.empty(0), torch.empty(0)
    elif len(false_positive_ids) == 1:
        # Only one false positive sample found
        return false_positive_ids[0].unsqueeze(0), false_positive_attn_masks[0].unsqueeze(0), false_positive_targets[0].unsqueeze(0)
    else:
        # Multiple false positive samples found
        return false_positive_ids, false_positive_attn_masks, false_positive_targets

# model = load_dma_experiment(run_id, dataset, dictionary_type, device)
gpu = 0
device = torch.device(f"cuda:{gpu}") 
model.to(device)    
# fids_unidentified = [ 457, 1070, 5097, 4628, 3344, 1058]
fp_cls_idx = 3213
# for fid in fids_unidentified:
#     print(f"File ID: {fid}")
#     context_strings = [" ".join(token) for token in L1_contexts[fid]["context_string"]]
#     context_string = "\n".join(context_strings)
#     print("Context String:\n", context_string)
#     print(f"Global Matrix Value: {global_mat[most_wrongly_predicted_cls_idx, fid]}")
#     print()

fp_sample_ids, fp_sample_attn_mask, fp_sample_targets = get_false_positive_samples(model, test_loader, fp_cls_idx,device= device)




In [ ]:
val, fp_fid = global_mat[fp_cls_idx,:].sort(descending=True)
i = 0
for fid in fp_fid[:300].tolist():
    i+=1
    if fid in L1_reduced_contexts:
        # print(L1_summaries[str(fid)])
        print(fid)
        context_strings = [" ".join(token) for token in L1_contexts[fid]["context_string"]]
        context_string = "\n".join(context_strings)
        print(context_string)
        # print(val[i])
        print("")
print(val[100])
# for cls_idx, count in wrong_predictions:
    # overall_wrong_classes.append(cls_idx)
# for cls_idx, count in false_positive_predictions:
#     if cls_idx not in overall_wrong_classes:
#         print(f"False Positives: Index: {cls_idx} Code: {label_transform.index2target[fid]}: {count}")

In [ ]:
import einops
def get_sample_f_ids(sample_ids, sample_attn_masks, model, device):
    h_sample = model.get_hidden_output(sample_ids.to(device), sample_attn_masks.to(device)).squeeze()
    f_sample = model.classifier_head.dictionary.encode(einops.rearrange(h_sample, "b s d -> (b s) d")).squeeze().squeeze()
    f_sample_activated = f_sample.max(dim=0).values
    f_val_sample, f_ids_sample = f_sample_activated.sort(descending=True)
    return f_val_sample.view(-1).cpu(), f_ids_sample.view(-1).cpu()

def get_overlapping_fids(tensor1, tensor2):
    # Convert the tensors to sets
    set1 = set(tensor1.view(-1).tolist())
    set2 = set(tensor2.view(-1).tolist())

    # Find the intersection of the sets
    overlapping_fids = set1.intersection(set2)
    # Convert the set of overlapping f_ids to a torch tensor
    overlapping_fids_tensor = torch.tensor(list(overlapping_fids), dtype=torch.int64)

    return overlapping_fids_tensor

def get_all_sample_f_ids(sample_ids, sample_attn_masks, model, device):
    all_f_samples = []
    for b in range(len(sample_ids)):
        h_sample = model.get_hidden_output(sample_ids[b].unsqueeze(0).to(device), sample_attn_masks[b].unsqueeze(0).to(device)).squeeze()
        f_sample = model.classifier_head.dictionary.encode(einops.rearrange(h_sample, "b s d -> (b s) d")).squeeze().squeeze()
        all_f_samples.append(f_sample)
    all_f_samples = torch.cat(all_f_samples, dim=0)
    f_sample_activated = f_sample.max(dim=0).values
    f_val_sample, f_ids_sample = f_sample_activated.sort(descending=True)
    return f_val_sample.view(-1).cpu(), f_ids_sample.view(-1).cpu()  

fp_f_val, fp_f_ids = get_all_sample_f_ids(fp_sample_ids, fp_sample_attn_mask, model, device)
print(fp_f_val[500])
fp_f_val_dict = {}
for i in range(len(fp_f_ids)):
    fp_f_val_dict[fp_f_ids[i].item()] = fp_f_val[i].item()

print(fp_f_ids[:10])
overlap = get_overlapping_fids(fp_fid[:2000], fp_f_ids[:2000])
print(fp_f_val[1000])
print(len(overlap))
print(val[1000])
i = 0



In [ ]:
for fid in overlap.tolist():
    i+=1
    if fid in L1_reduced_contexts:
        # print(L1_summaries[str(fid)])
        associative_score = global_mat[fp_cls_idx, fid].item()
        feature_score = fp_f_val_dict[fid]
        if associative_score * feature_score > 2.0:
            print(fid)
            context_strings = [" ".join(token) for token in L1_contexts[fid]["context_string"]]
            context_string = "\n".join(context_strings)
            print(context_string)
            # print("associative score:", global_mat[fp_cls_idx, fid].item())
            # print("magnitude of f:", fp_f_val_dict[fid])
            print("")
    else:
        print("")

In [ ]:
relevant_fids = []
for fid in overlap.tolist():
    if global_mat[fp_cls_idx, fid].item() > 0.3:
        relevant_fids.append(fid)
print(len(relevant_fids))

for fid in relevant_fids:
    i+=1
    print(fid)
    if fid in L1_reduced_contexts:
        # print(L1_summaries[str(fid)])
        # print(fid)
        context_strings = [" ".join(token) for token in L1_contexts[fid]["context_string"]]
        context_string = "\n".join(context_strings)
        print(context_string)
        # print("associative score:", global_mat[fp_cls_idx, fid].item())
        # print("magnitude of f:", fp_f_val_dict[fid])
        print("")
    else:
        print("")

In [ ]:
# compute an F1 score for the model after reweighing to observe effects on other classes
#
def reweighted_prediction(sample_token_ids, sample_attn_masks, sample_targets, cls_idx, model :SparsePLMICDClassifierModule, fids, device):
    # create fid weight matrix with the same shape as the global matrix
    fid_addition_matrix = torch.zeros(model.classifier_head.icd_memory.data.shape)
    # for fid, count in fids:
    #     fid_addition_matrix[0,cls_idx, fid] = 0  # high weight > 1.0
    # fids = [5188, 4443, 345, 1558, 802, 6069, 4917, 1590, 5550, 2457, 4504, 3623, 1713, 4406, 3773, 3394, 4333, 4339, 1910, 5545, 5044, 1391, 1665, 1288, 3220, 4630, 3607, 4504, 4508, 3623, 3773, 5550, 1713, 563, 2612, 4406, 5812, 1919, 4156, 4668, 1343, 3394,  4677, 5188, 1730, 5831,1873,1877, 1245, 734, 489, 4333, 3693, 1391, 4339]
    print(len(fids))
    for fid in fids:
        fid_addition_matrix[0, cls_idx, fid] = - model.classifier_head.icd_memory.data[0, cls_idx, fid] # high weight > 1.0, negative respective weights in the fid list 

    direct_related_fid = 5387
    fid_addition_matrix[0,cls_idx, direct_related_fid] = 3.0
    # semi_related_fids = [5297, 5836, 3311, 1775, 3320, 764, 4357, 4368, 1306, 4896, 3920, 1889, 907, 4500, 5545, 3015, 5066, 988]
    # semi_related_fids = [5297, 5836,  764, 4357, 4368, 1306, 4896, 3920, 907, 4500, 5545, 3015, 5066, 988]
    # for fid in semi_related_fids:
    #     fid_addition_matrix[0,cls_idx, fid] = 1.0
    # fid_addition_matrix[0, cls_idx, :] = - model.classifier_head.icd_memory.data[0, cls_idx, :]
    # fid_addition_matrix[0, cls_idx, 5188] = -1.4
    # fid_addition_matrix[0, cls_idx, 4443] = -2.0266
    # fid_addition_matrix[0, cls_idx, 345] = -1.4272
    # fid_addition_matrix[0, cls_idx, 1558] = -1.3

    # fid_addition_matrix[0,cls_idx, 4817] = 6 # what if we just upweight all the fids and force predict more!
    
    nCorrect = 0 # nCorrect of cls idx
    ogCorrect = 0
    y_true = []
    y_og_prob = []
    y_prob = []
    for i in range(len(sample_token_ids)):
        token_ids = sample_token_ids[i].unsqueeze(0)
        attn_mask = sample_attn_masks[i].unsqueeze(0)
        h_out = model.get_hidden_output(token_ids.to(device), attn_mask.to(device))
        h_out = rearrange(h_out, "b n s d -> b (n s) d")
        # vector_to_append = 11 * model.classifier_head.dictionary.get_dictionary_matrix()[4817]
        # vector_to_append = vector_to_append.unsqueeze(0).expand(h_out.size(0), -1)
        # vector_to_append = vector_to_append.unsqueeze(1)
        # h_out = torch.cat((h_out, vector_to_append), dim=1)
  
        og_logits, _ = model(token_ids.to(device), attn_mask.to(device))
        logits = model.get_modified_forward(h_out, fid_addition_matrix.to(device))
        print(logits[:,cls_idx])
        # print(og_logits.shape)
        # print(sample_targets[i].shape)
        # print(logits.shape)
        y_true.append(sample_targets[i].numpy())
        y_og_prob.append(torch.sigmoid(og_logits).cpu().detach().squeeze().numpy())
        y_prob.append(torch.sigmoid(logits).cpu().detach().squeeze().numpy())
        og_pred = (torch.sigmoid(og_logits) > 0.3).cpu().int()
        pred = (torch.sigmoid(logits) > 0.3).cpu().int()
        # print(pred.shape)
        # print(sample_targets[i].shape)
        print(pred[0, cls_idx].item(), sample_targets[i][cls_idx].item())
        if pred[0, cls_idx].item() == sample_targets[i][cls_idx].int().item():
            nCorrect += 1
            print("nCorrect:", nCorrect, "out of:", i + 1)
        if og_pred[0, cls_idx].item() == sample_targets[i][cls_idx].int().item():
            ogCorrect+=1
            print("Original Correct", ogCorrect, "out of:", i + 1)

    y_true = np.array(y_true)
    y_prob = np.array(y_prob)
    y_og_prob = np.array(y_og_prob)
    modified_metrics = multilabel_metrics_fn(y_true, y_prob, metrics=["f1_macro", "f1_micro"])
    print("Modified:", modified_metrics)
    print("nCorrect:", nCorrect, "out of:", len(sample_targets))
    og_metrics = multilabel_metrics_fn(y_true, y_og_prob, metrics=["f1_macro", "f1_micro"])
    print("Original:", og_metrics)
    print("Original Correct", ogCorrect, "out of:",len(sample_targets))

    return nCorrect / len(sample_token_ids)

# all of my identified ones (that potentially affect it) -> doesn't really work all that well.
# relevant_fids = [5188, 4443, 345, 1558, 802, 6069, 4917, 1590, 5550, 2457, 4504, 3623, 1713, 4406, 3773, 3394, 4333, 4339, 1910, 5545, 5044, 1391, 1665, 1288, 3220, 4630, 3607, 4504, 4508, 3623, 3773, 5550, 1713, 563, 2612, 4406, 5812, 1919, 4156, 4668, 1343, 3394,  4677, 5188, 1730, 5831,1873,1877, 1245, 734, 489, 4333, 3693, 1391, 4339, 4630, 3607, 1049, 2084, 5669, 3623, 2096, 2612, 1594, 4668, 3132, 65, 5188, 4677, 3693, 4722, 1138, 1665, 3213, 1713, 6141, 3773, 1730, 1245, 2274, 4333, 4339, 1288, 4890, 4406, 1846, 1343, 321, 3394, 2892, 1873, 1877, 1391, 4504, 4508, 5550, 6069, 2495, 4546, 3539, 489, 3573, 3067]
# all of them lol
# all of claude identified
# relevant_fids = [4630, 3607, 1049, 2084, 5669, 3623, 2096, 563, 2612, 1594, 4156, 4668, 3132, 65, 5188, 4677, 2129, 3693, 4722, 1138, 3704, 127, 1153, 1665, 3213, 1713, 691, 5812, 6141, 3773, 1730, 5831, 4827, 734, 1245, 2274, 3818, 4333, 3311, 1775, 4339, 244, 3320, 1788, 2308, 4358, 1288, 4368, 791, 4890, 3876, 2864, 820, 4406, 1846, 1343, 321, 3394, 1353, 2892, 1873, 2899, 1877, 1889, 1391, 1905, 1910, 4984, 1407, 1919, 4500, 4504, 4508, 4002, 937, 5545, 2479, 5550, 5044, 6069, 3518, 2495, 2497, 4546, 5060, 2501, 5066, 3539, 6104, 489, 1008, 3573, 3067]
# relevant_fids = [4630, 3607, 1049, 2084, 5669, 3623, 2096, 2612, 1594, 4668, 3132, 65, 5188, 4677, 2129, 3693, 4722, 1138, 3704, 1153, 1665, 3213, 1713, 691, 5812, 6141, 3773, 1730, 1245, 2274, 3818, 4333, 4339, 1288, 4890, 4406, 1846, 1343, 321, 3394, 2892, 1873, 1877, 4984, 1391, 4504, 4508, 5550, 6069, 2495, 4546, 489, 3573, 3067]
# relevant_fids = [1665, 1288, 4630, 3607, 4504, 4508, 3623, 3773, 5550, 1713, 4406, 1919, 4668, 1343, 3394, 4677, 5188, 1877, 1245, 489, 4333, 3693, 1391, 4722, 4339, 1138, 1905, 1910]
# relevant_fids = [2084, 2096, 65, 4244, 4267, 2274, 4333, 4339, 4406, 4504, 410, 4508, 2495, 2497, 2508, 489, 4630, 2612, 4668, 4677, 4722, 4725, 791, 4890, 2892, 2895, 942, 3030, 5099, 3067, 1049, 5188, 3146, 1138, 3213, 1245, 3318, 1288, 1665, 1288, 4630, 3607, 4504, 4508, 3623, 3773, 5550, 1713, 4406, 1919, 4668, 1343, 3394, 4677, 5188, 1877, 1245, 489, 4333, 3693, 1391, 4722, 4339, 1138, 1905, 1910]
relevant_fids = [5188, 4443, 345, 1558, 802, 6069, 4917] # best one lmao by itself
# relevant_fids = [4333, 4339, 4358, 4406, 4504, 2479, 2501, 2612, 4722, 2864, 5044, 5060, 5188, 3394, 1391, 5545, 5550, 5556, 3518, 3623, 3693, 3704, 1665, 1713, 3773, 1873, 1910, 6069]
# relevant_fids = [5188, 4443, 345, 1558, 802, 6069, 4917, 1590, 5550, 2457, 4504, 3623, 1713, 4406]
# relevant_fids = [127, 410, 489, 563, 691, 791, 820, 937, 1008, 1138, 1146, 1153, 1245, 1288, 1353, 1391, 1407, 1584, 1665, 1713, 1730, 1775, 1788, 1846, 1873, 1877, 1889, 1905, 1910, 2129, 2274, 2308, 2479, 2501, 2612, 2864, 2899, 3067, 3123, 3132, 3311, 3320, 3394, 3518, 3573, 3607, 3623, 3693, 3704, 3773, 3818, 3828, 4002, 4019, 4156, 4244, 4333, 4339, 4358, 4368, 4406, 4500, 4504, 4508, 4546, 4630, 4677, 4722, 5060, 5066, 5099, 5188, 5556, 5812, 5831, 5879, 6069, 6104] # supposedly this one has the greatest effect on false positives.
reweighted_prediction(fp_sample_ids, fp_sample_attn_mask, fp_sample_targets, fp_cls_idx, model, relevant_fids, device)
print(len(fp_sample_ids))

In [ ]:
from einops import rearrange
import numpy as np
def reweighted_prediction_across_all(test_loader, cls_idx, model: SparsePLMICDClassifierModule, fids, device):
    # create fid weight matrix with the same shape as the global matrix
    fid_addition_matrix = torch.zeros(model.classifier_head.icd_memory.data.shape)

    for fid in fids:
        fid_addition_matrix[0, cls_idx, fid] = - model.classifier_head.icd_memory.data[0, cls_idx, fid]  # high weight > 1.0, negative respective weights in the fid list
    # fid_addition_matrix[0, cls_idx, :] = -model.classifier_head.icd_memory.data[0, cls_idx, fid] 
    # direct_related_fid = 5387
    # fid_addition_matrix[0, cls_idx, direct_related_fid] = 8.0
    # my_last_try = [3311, 1775, 3320, 4357, 4368, 4500, 3015, 5066, 2129, 5297, 5836, 3311, 1775, 3320, 764, 4357, 4368, 1306, 4896, 3920, 1889, 907, 4500, 5545, 3015, 5066, 988]
    # for fid in my_last_try:
    #     fid_addition_matrix[0,cls_idx, fid] = 8.0


    nCorrect = 0  # nCorrect of cls idx
    ogCorrect = 0
    nFalsePositive = 0
    nFalseNegative = 0

    ogFalsePositive = 0
    ogFalseNegative = 0
    y_true = []
    y_og_prob = []
    y_prob = []

    with torch.no_grad():
        for batch in test_loader:
            token_ids, targets, attn_mask = batch.data, batch.targets, batch.attention_mask

            h_out = model.get_hidden_output(token_ids.to(device), attn_mask.to(device))
            h_out = rearrange(h_out, "b n s d -> b (n s) d")

            og_logits, _ = model(token_ids.to(device), attn_mask.to(device))
            logits = model.get_modified_forward(h_out, fid_addition_matrix.to(device))

            y_true.append(targets)
            y_og_prob.append(torch.sigmoid(og_logits).cpu().detach())
            y_prob.append(torch.sigmoid(logits).cpu().detach())

            og_pred = (torch.sigmoid(og_logits) > 0.3).cpu().int()
            pred = (torch.sigmoid(logits) > 0.3).cpu().int()

            nCorrect += (pred[:, cls_idx] == targets[:, cls_idx]).sum().item()
            ogCorrect += (og_pred[:, cls_idx] == targets[:, cls_idx]).sum().item()

            # Compute false positives and false negatives
            nFalsePositive += ((pred[:, cls_idx] == 1) & (targets[:, cls_idx] == 0)).sum().item()
            nFalseNegative += ((pred[:, cls_idx] == 0) & (targets[:, cls_idx] == 1)).sum().item()
            ogFalsePositive += ((og_pred[:, cls_idx] == 1) & (targets[:, cls_idx] == 0)).sum().item()
            ogFalseNegative += ((og_pred[:, cls_idx] == 0) & (targets[:, cls_idx] == 1)).sum().item()

    y_true = torch.cat(y_true, dim=0).numpy()
    y_prob = torch.cat(y_prob, dim=0).numpy()
    y_og_prob = torch.cat(y_og_prob, dim=0).numpy()

    modified_metrics = multilabel_metrics_fn(y_true, y_prob, metrics=["f1_macro", "f1_micro"])
    print("Modified:", modified_metrics)
    print("nCorrect:", nCorrect, "out of:", len(test_loader) * 8)
    print("False Positives:", nFalsePositive)
    print("False Negatives:", nFalseNegative)

    og_metrics = multilabel_metrics_fn(y_true, y_og_prob, metrics=["f1_macro", "f1_micro"])
    print("Original:", og_metrics)
    print("Original Correct", ogCorrect, "out of:", len(test_loader) * 8)
    print("gg FP:", ogFalsePositive)
    print("og FN:", ogFalseNegative)
    return nCorrect / len(test_loader), nFalsePositive, nFalseNegative

# relevant_fids = [5188, 4443, 345, 1558, 802, 6069, 4917, 1590, 5550, 2457, 4504, 3623, 1713, 4406, 3773, 3394, 4333, 4339, 1910, 5545, 5044, 1391, 1665, 1288, 3220, 4630, 3607, 4504, 4508, 3623, 3773, 5550, 1713, 563, 2612, 4406, 5812, 1919, 4156, 4668, 1343, 3394,  4677, 5188, 1730, 5831,1873,1877, 1245, 734, 489, 4333, 3693, 1391, 4339, 4630, 3607, 1049, 2084, 5669, 3623, 2096, 2612, 1594, 4668, 3132, 65, 5188, 4677, 3693, 4722, 1138, 1665, 3213, 1713, 6141, 3773, 1730, 1245, 2274, 4333, 4339, 1288, 4890, 4406, 1846, 1343, 321, 3394, 2892, 1873, 1877, 1391, 4504, 4508, 5550, 6069, 2495, 4546, 3539, 489, 3573, 3067]
# relevant_fids = [1665, 1288, 4630, 3607, 4504, 4508, 3623, 3773, 5550, 1713, 4406, 1919, 4668, 1343, 3394, 4677, 5188, 1877, 1245, 489, 4333, 3693, 1391, 4722, 4339, 1138, 1905, 1910]
relevant_fids = [5188, 4443, 345, 1558, 802, 6069, 4917] # this one isn't bad considering it only uses the top whatever, if this last try doesn't work, this is what we'll report hilariously.
# relevant_fids = [127, 410, 489, 563, 691, 791, 820, 937, 1008, 1138, 1146, 1153, 1245, 1288, 1353, 1391, 1407, 1584, 1665, 1713, 1730, 1775, 1788, 1846, 1873, 1877, 1889, 1905, 1910, 2129, 2274, 2308, 2479, 2501, 2612, 2864, 2899, 3067, 3123, 3132, 3311, 3320, 3394, 3518, 3573, 3607, 3623, 3693, 3704, 3773, 3818, 3828, 4002, 4019, 4156, 4244, 4333, 4339, 4358, 4368, 4406, 4500, 4504, 4508, 4546, 4630, 4677, 4722, 5060, 5066, 5099, 5188, 5556, 5812, 5831, 5879, 6069, 6104]
# relevant_fids = [5188, 4443, 345, 1558, 802, 6069, 4917, 1590, 5550, 2457, 4504, 3623, 1713, 4406] # lowers f1
# relevant_fids = [4333, 4339, 4358, 4406, 4504, 2479, 2501, 2612, 4722, 2864, 5044, 5060, 5188, 3394, 1391, 5545, 5550, 5556, 3518, 3623, 3693, 3704, 1665, 1713, 3773, 1873, 1910, 6069]
# relevant_fids = [4443, 5017, 5683, 4406,2568]
# relevant_fids = [5188, 4443, 345, 1558, 802, 6069, 4917, 4443, 5017, 5683, 4406, 2568]
gpu = 0
device = torch.device(f"cuda:{gpu}") 
model.to(device)    
fp_cls_idx = 3213
reweighted_prediction_across_all(test_loader, fp_cls_idx, model, relevant_fids, device)


In [ ]:



def get_ablation_results_modified_forward(dataloader, model: SparsePLMICDClassifierModule, device, n_batches=10, quantile=0.95, use_precise=True):
    iterator = iter(dataloader)
    if n_batches is None:
        n_batches = len(dataloader)  # use all if None
    fid_addition_matrix = torch.zeros(model.classifier_head.icd_memory.data.shape)
    # for fid, count in fids:
    #     fid_addition_matrix[0,cls_idx, fid] = 0  # high weight > 1.0
    # fids = [5188, 4443, 345, 1558, 802, 6069, 4917, 1590, 5550, 2457, 4504, 3623, 1713, 4406, 3773, 3394, 4333, 4339, 1910, 5545, 5044, 1391, 1665, 1288, 3220, 4630, 3607, 4504, 4508, 3623, 3773, 5550, 1713, 563, 2612, 4406, 5812, 1919, 4156, 4668, 1343, 3394,  4677, 5188, 1730, 5831,1873,1877, 1245, 734, 489, 4333, 3693, 1391, 4339]
    print(len(fids))
    for fid in fids:
        fid_addition_matrix[0, cls_idx, fid] = - model.classifier_head.icd_memory.data[0, cls_idx, fid] # high weight > 1.0, negative respective weights in the fid list 

    total_notes = 0
    metrics = {
        'p_delta_laat_not_targets': [],
        'p_delta_dictionary_not_targets': [],
        'p_delta_top_cls_drop_laat': [],
        'p_delta_top_cls_drop_dict': []
    }

    for i in range(n_batches):
        batch = next(iterator)
        tokens, targets, attn_mask = batch.data, batch.targets, batch.attention_mask
        b, c, t = tokens.shape

        for note in range(b):
            note_tokens = tokens[note].unsqueeze(0)  # keep the batch size
            note_attn_mask = attn_mask[note].unsqueeze(0)

            # fwd pass, only get ground truth classes
            og_hidden_out = model.get_hidden_output(note_tokens.to(device), note_attn_mask.to(device))
            og_hidden_out = og_hidden_out.view(1, c * t, -1)
            with torch.no_grad():
                og_pred = torch.sigmoid(model.get_reconstructed_logits(og_hidden_out))
            og_top_softmax, top_cls_idx = og_pred.max(dim=1)  # get back the top class for each example

            # create reverse one-hot vector for none-top class
            not_note_targets = torch.ones(og_pred.shape)
            not_note_targets[:, top_cls_idx] = 0  # set the top class to 0, to ignore

            # get attn
            label_attn = model.classifier_head.attn.detach()
            token_attn_top_cls = label_attn.squeeze()[top_cls_idx]
            threshold = torch.quantile(token_attn_top_cls, quantile, dim=1)
            not_relevant_tokens = token_attn_top_cls < threshold
            token_ablated_h_out = not_relevant_tokens.view(1, c * t, 1) * og_hidden_out  # keep only not_relevant_tokens

            p_ablated = torch.sigmoid(model.get_reconstructed_logits(token_ablated_h_out))
            p_delta_top_cls_drop_laat = (og_pred - p_ablated)[:, top_cls_idx].item()
            delta_p_not_targets = not_note_targets.to(p_ablated.device) * (og_pred - p_ablated).abs()
            p_delta_laat_not_targets = delta_p_not_targets.sum().item()

            relevant_tokens = token_attn_top_cls >= threshold
            relevant_tokens = torch.nonzero(relevant_tokens.view(-1))

            f_acts, x_recon, aenc_loss_dict = model.dictionary(og_hidden_out.squeeze())
            f_acts = f_acts.squeeze()
            most_relevant_f_acts = f_acts[relevant_tokens.squeeze()].squeeze()  # should be relevant tokens x f
            most_relevant_fids = torch.nonzero(most_relevant_f_acts.max(dim=0).values).squeeze()  # non-zero are the most relevant f_ids
            f_ablated_probs = torch.sigmoid(model.get_ablated_f_probs(og_hidden_out, most_relevant_fids, top_cls_idx, use_precise=use_precise))
            p_most_relevant_to_cls = og_pred - f_ablated_probs
            p_delta_top_cls_drop_dict = p_most_relevant_to_cls[0, top_cls_idx].item()
            deltas_not_targets_f_ablated = not_note_targets.to(p_most_relevant_to_cls.device) * p_most_relevant_to_cls.abs()
            p_delta_dictionary_not_targets = deltas_not_targets_f_ablated.sum().item()

            total_notes += 1
            print(f"Total Notes: {total_notes}")

            metrics['p_delta_laat_not_targets'].append(p_delta_laat_not_targets)
            metrics['p_delta_dictionary_not_targets'].append(p_delta_dictionary_not_targets)
            metrics['p_delta_top_cls_drop_laat'].append(p_delta_top_cls_drop_laat)
            metrics['p_delta_top_cls_drop_dict'].append(p_delta_top_cls_drop_dict)

    # Compute means and standard deviations
    for metric, values in metrics.items():
        mean = sum(values) / total_notes
        variance = sum((x - mean) ** 2 for x in values) / (total_notes - 1)
        std_dev = torch.sqrt(torch.tensor(variance)).item()
        metrics[metric] = {'mean': mean, 'std_dev': std_dev}

    print(f"Metrics: {metrics}")
    return metrics

In [ ]:
mag, wrongly_predicted_f_idxs = global_mat[most_wrongly_predicted_cls_idx].sort(descending=True)
for fid in wrongly_predicted_f_idxs[:50].tolist():
    print(f"File ID: {fid}")
    context_strings = [" ".join(token) for token in L1_contexts[fid]["context_string"]]
    context_string = "\n".join(context_strings)
    print("Context String:\n", context_string)
    print(f"Global Matrix Value: {global_mat[most_wrongly_predicted_cls_idx, fid]}")
    print()

In [ ]:
diabetes_feature = 2353	
print(global_mat.shape)
print(L1_summaries[str(diabetes_feature)])
print(global_mat[:,diabetes_feature].sort(descending=True))
print(label_transform.index2target[1866])
print(icd9cm.lookup(label_transform.index2target[1866]))
diabetes_cls_idx = 1866
val, diabetes_f_idxs = global_mat[diabetes_cls_idx,:].sort(descending=True)
print("----------")
print(global_mat[diabetes_cls_idx,:].sort(descending=True))
for fid in diabetes_f_idxs:
    print(fid)
    if str(fid.item()) in L1_summaries:
        print(L1_summaries[str(fid.item())])
        print(L1_contexts[fid.item()]["context_string"])

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite

feature_labels = {}
for i in range(global_mat.shape[1]):
    if str(i) in L1_summaries:
        summary = L1_summaries[str(i)]
        feature_labels[i] = summary
    else:
        feature_labels[i] = f"{i}"

class_labels = {}
for cls_idx in range(global_mat.shape[0]):
    cls_code = label_transform.index2target[cls_idx]
    cls_desc = ""
    if cls_code in icd9cm:
        cls_desc = icd9cm.lookup(cls_code)
    elif cls_code in icd9proc:
        cls_desc = icd9proc.lookup(cls_code)
    class_labels[cls_idx] = cls_desc


graph_global_mat = (global_mat > 1) * global_mat
graph_global_mat = graph_global_mat.cpu().numpy()

# Create a bipartite graph
# plt.figure(figsize=(100,100))

# B = nx.Graph()

# Add nodes with bipartite attribute
# top_nodes = range(graph_global_mat.shape[0])
# bottom_nodes = range(graph_global_mat.shape[0], graph_global_mat.shape[0] + graph_global_mat.shape[1])
# B.add_nodes_from(top_nodes, bipartite=0)
# B.add_nodes_from(bottom_nodes, bipartite=1)

# # Add edges based on the graph_global_mat matrix
# edges = [(i, j + graph_global_mat.shape[0]) for i in range(graph_global_mat.shape[0]) for j in range(graph_global_mat.shape[1]) if graph_global_mat[i][j] > 0]
# B.add_edges_from(edges)

# # Set node colors for top and bottom nodes
# top_nodes = {n for n, d in B.nodes(data=True) if d['bipartite'] == 0}
# bottom_nodes = set(B) - top_nodes
# node_colors = ['lightblue' if n in top_nodes else 'lightgreen' for n in B.nodes()]

# # Create node labels using the provided class_labels and feature_labels
# node_labels = {n: class_labels[n] for n in top_nodes}
# node_labels.update({n: feature_labels[n - graph_global_mat.shape[0]] for n in bottom_nodes})

# # Draw the bipartite graph
# pos = nx.bipartite_layout(B, top_nodes)
# nx.draw_networkx(B, pos, node_color=node_colors, with_labels=True, labels=node_labels, 
#                  font_size=8, font_weight='bold', node_size=500)

# # Show the plot
# plt.axis('off')
# plt.savefig("graph_test.png")

In [ ]:
diabetes_cls_idx = 1866
val, diabetes_f_idxs = global_mat[diabetes_cls_idx,:].sort(descending=True)
print("----------")
print(global_mat[diabetes_cls_idx,:].sort(descending=True))
for fid in diabetes_f_idxs:
    print(fid)
    if str(fid.item()) in L1_summaries:
        print(L1_summaries[str(fid.item())])
        print(L1_contexts[fid.item()]["context_string"])


# Create a subgraph with the node of interest and its neighbors
node_of_interest = diabetes_cls_idx
neighbors = list(B.neighbors(node_of_interest))
subgraph = B.subgraph([node_of_interest] + neighbors)
# Set the figure size
plt.figure(figsize=(10, 10))  # Adjust the width and height as needed

# Set node colors for the subgraph
node_colors = ['lightblue' if n == node_of_interest else 'lightgreen' for n in subgraph.nodes()]

# Create node labels for the subgraph
subgraph_node_labels = {n: node_labels[n] for n in subgraph.nodes()}

# Draw the subgraph
pos = nx.bipartite_layout(subgraph, [node_of_interest])
nx.draw_networkx(subgraph, pos, node_color=node_colors, with_labels=True, labels=subgraph_node_labels,
                 font_size=8, font_weight='bold', node_size=400)

# Show the plot
plt.axis('off')
plt.tight_layout()  # Adjust the spacing between subplots
plt.show()

In [ ]:
import squarify
import matplotlib.pyplot as plt
import seaborn as sns

def plot_treemap(code, top_concepts, sizes, labels, color_palette='deep'):
    # Set the figure size and create a new figure
    plt.figure(figsize=(12, 8))
    
    # Create a color palette using Seaborn
    colors = sns.color_palette(color_palette, len(sizes))
    
    # Create the treemap using squarify with the specified color palette
    squarify.plot(sizes=sizes, label=labels, color=colors, alpha=0.8)
    
    # Add a title to the plot
    plt.title(f"Treemap of Top-N Medical Concepts for Code: {code}")
    
    # Remove the axis
    plt.axis('off')
    
    # Display the plot
    plt.show()

# Example usage
code = "ICD-10-CM E11"
top_concepts = ["Type 2 Diabetes Mellitus", "Insulin Resistance", "Hyperglycemia", "Obesity", "Diabetic Nephropathy"]
sizes = [100, 80, 60, 50, 40]
labels = [f"{concept}\n(Size: {size})" for concept, size in zip(top_concepts, sizes)]

plot_treemap(code, top_concepts, sizes, labels, color_palette='pastel')

In [ ]:
import squarify
import matplotlib.pyplot as plt
import re
import seaborn as sns
import textwrap
from pyhealth.medcode import InnerMap
# 2353 diabetes, 5879 for infection sepsis
diabetes_feature = 2353	
# diabetes_feature = 5879
# basically given an interpretable dictionary feature
# give me 5 highly related medical codes
# for each of those 5, get the 5 top dictionary features using our global_mat
# returns a list of codes
# returns a list of lists of top concepts for each code
# returns a list of lists of sizes for each top concepts
def remove_common_pattern_phrases(text):
    # Define the regular expression pattern
    pattern = r'(?i)(is the common pattern|The common pattern is|The common pattern observed is|are the common pattern|is a common complication in these cases|are common patterns observed|with liver biopsy procedure.|, likely Down syndrome \(Trisomy 21\), and associated genetic counseling.|Complications related to|for resection of brain masses, along with supportive measures.|, including agitation, hallucinations, and complications like pancreatitis.|for osteoarthritis and other conditions.|is a common theme.|placement, perforation, and metastatic cancer.)'
    # Replace the matched phrases with an empty string
    filtered_text = re.sub(pattern, '', text)
    return filtered_text

def get_top_concepts_for_code_given_feature(global_mat, idx, icd9proc, icd9cm, label_transform, n=5):
    cls_val, cls_idxs = global_mat[:,idx].sort(descending=True)
    top_codes = []
    for i in range(n):
        code = label_transform.index2target[cls_idxs[i].item()]
        code_desc = ""
        if code in icd9proc:
            code_desc = icd9proc.lookup(code)
        elif code in icd9cm:
            code_desc = icd9cm.lookup(code)
        code = f"{code}: {code_desc}"
        top_codes.append(code)
    top_concepts_list = []
    sizes_list = []
    for i in range(n):
        top_concepts = []
        sizes = []
        cls_f_val, cls_f_idxs = global_mat[cls_idxs[i],:].sort(descending=True)
        for j in range(10):
            fid = cls_f_idxs[j]
            if str(fid.item()) in L1_summaries:
                top_concepts.append(f"{fid.item()}: " + remove_common_pattern_phrases(L1_summaries[str(fid.item())]) + f", {cls_f_val[j].item():.2f}")
            else:
                top_concepts.append(f"{fid.item()}: Unidentfiable, {cls_f_val[j].item():.2f}")
            sizes.append(cls_f_val[j].item())
        top_concepts_list.append(top_concepts)
        sizes_list.append(sizes)
    return top_codes, top_concepts_list, sizes_list


def wrap_labels(labels, wrap_width=20):
    wrapped_labels = []
    for label in labels:
        wrapped_label = textwrap.fill(label, width=wrap_width)
        wrapped_labels.append(wrapped_label)
    return wrapped_labels

def wrap_title(title, max_words=5):
    words = title.split()
    if len(words) > max_words:
        wrapped_title = ' '.join(words[:max_words]) + '\n' + ' '.join(words[max_words:])
    else:
        wrapped_title = title
    return wrapped_title

def scale_text_sizes(sizes, min_size=12, max_size=24):
    normalized_sizes = (sizes - np.min(sizes)) / (np.max(sizes) - np.min(sizes))
    text_sizes = normalized_sizes * (max_size - min_size) + min_size
    return text_sizes

# def plot_multiple_treemaps(codes, top_concepts_list, sizes_list, labels_list, path, color_palette='deep', size=60, min_text_size=12, max_text_size=24):
#     # Set the figure size and create a new figure with a specific aspect ratio
#     fig_width = size
#     fig_height = fig_width / len(codes)
#     fig, axs = plt.subplots(nrows=1, ncols=len(codes), figsize=(fig_width, fig_height))
    
#     # Create a color palette using Seaborn
#     colors = sns.color_palette(color_palette, len(sizes_list[0]))
    
#     # Iterate over each medical code and its associated data
#     for i, (code, top_concepts, sizes, labels) in enumerate(zip(codes, top_concepts_list, sizes_list, labels_list)):
#         # Wrap the labels to fit within the subsquares
#         wrapped_labels = wrap_labels(labels)
        
#         # Scale the text sizes based on the sizes
#         text_sizes = scale_text_sizes(np.array(sizes), min_text_size, max_text_size)
        
#         # Create the treemap using squarify with padding
#         ax = squarify.plot(sizes=sizes,  alpha=0.8, ax=axs[i], color=colors, pad=True)
        
#         # Add labels to the rectangles with scaled text sizes
#         for rect, label, text_size in zip(ax.patches, wrapped_labels, text_sizes):
#             x, y = rect.get_xy()
#             dx, dy = rect.get_width(), rect.get_height()
#             ax.text(x + dx / 2, y + dy / 2, label, fontsize=text_size, wrap=True, weight='bold', ha='center', va='center')
        
#         # Wrap the subplot title if it exceeds the specified number of words
#         wrapped_title = wrap_title(f"ICD9 {code}", max_words=5)
        
#         # Add the wrapped title to each subplot with bold text
#         ax.set_title(wrapped_title, fontsize=16, fontweight='bold')
        
#         # Remove the axis for each subplot
#         ax.axis('off')
        
#         # Set equal aspect ratio for each subplot
#         ax.set_aspect('equal')
    
#     # Adjust the spacing between subplots
#     plt.subplots_adjust(wspace=0.1, hspace=0.1)
#     plt.savefig(path)
#     # Display the plot
#     plt.show()

def plot_multiple_treemaps(codes, top_concepts_list, sizes_list, labels_list, path, color_palette='deep', size=60, min_text_size=12, max_text_size=24):
    # Set the figure size and create a new figure with a specific aspect ratio
    fig_width = size
    fig_height = fig_width / len(codes)
    fig, axs = plt.subplots(nrows=1, ncols=len(codes), figsize=(fig_width, fig_height))
    
    # Adjust the subplot parameters to remove extra white spacing
    plt.subplots_adjust(wspace=0.1, hspace=0.1, left=0, right=1, bottom=0, top=1)
    
    # Create a color palette using Seaborn
    colors = sns.color_palette(color_palette, len(sizes_list[0]))
    
    # Iterate over each medical code and its associated data
    for i, (code, top_concepts, sizes, labels) in enumerate(zip(codes, top_concepts_list, sizes_list, labels_list)):
        # Wrap the labels to fit within the subsquares
        wrapped_labels = wrap_labels(labels)
        
        # Scale the text sizes based on the sizes
        text_sizes = scale_text_sizes(np.array(sizes), min_text_size, max_text_size)
        
        # Create the treemap using squarify with padding
        ax = squarify.plot(sizes=sizes, alpha=0.8, ax=axs[i], color=colors, pad=True)
        
        # Add labels to the rectangles with scaled text sizes
        for rect, label, text_size in zip(ax.patches, wrapped_labels, text_sizes):
            x, y = rect.get_xy()
            dx, dy = rect.get_width(), rect.get_height()
            ax.text(x + dx / 2, y + dy / 2, label, fontsize=text_size, wrap=True, weight='bold', ha='center', va='center')
        
        # Wrap the subplot title if it exceeds the specified number of words
        wrapped_title = wrap_title(f"ICD9 {code}", max_words=5)
        
        # Add the wrapped title to each subplot with bold text
        ax.set_title(wrapped_title, fontsize=34, fontweight='bold')
        
        # Remove the axis for each subplot
        ax.axis('off')
        
        # Set equal aspect ratio for each subplot
        ax.set_aspect('equal')
    
    # Save the figure with tight bounding box
    plt.savefig(path, bbox_inches='tight')
    
    # Display the plot
    plt.show()

diabetes_feature = 2353	
icd9proc = InnerMap.load("ICD9PROC")
icd9cm = InnerMap.load("ICD9CM")
top_codes, top_concepts_list, sizes_list = get_top_concepts_for_code_given_feature(global_mat, diabetes_feature, icd9proc, icd9cm, label_transform, n=3)
plot_multiple_treemaps(top_codes, top_concepts_list, sizes_list, top_concepts_list, path="figs/diabetes_mapping.png", color_palette="pastel", min_text_size=24, max_text_size=36)

In [ ]:
sepsis_feature = 5879
top_codes, top_concepts_list, sizes_list = get_top_concepts_for_code_given_feature(global_mat, sepsis_feature, icd9proc, icd9cm, label_transform, n=3)
plot_multiple_treemaps(top_codes, top_concepts_list, sizes_list, top_concepts_list, path="figs/sepsis_mapping.png", color_palette="pastel", size=60, min_text_size=24, max_text_size=36)

In [ ]:
chest_tube_feature = 4817
top_codes, top_concepts_list, sizes_list = get_top_concepts_for_code_given_feature(global_mat, chest_tube_feature, icd9proc, icd9cm, label_transform, n=5)
plot_multiple_treemaps(top_codes, top_concepts_list, sizes_list, top_concepts_list, path="figs/chest_tube_cath_mapping.png", color_palette="pastel", size=60, min_text_size=10, max_text_size=22)

In [ ]:
platelet_feature = 5556
top_codes, top_concepts_list, sizes_list = get_top_concepts_for_code_given_feature(global_mat, platelet_feature, icd9proc, icd9cm, label_transform, n=3)
plot_multiple_treemaps(top_codes, top_concepts_list, sizes_list, top_concepts_list, path="figs/platelet_related_mapping.png", color_palette="pastel", size=10, min_text_size=24, max_text_size=36)

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px

def plot_multiple_stacked_bar(codes, top_concepts_list, sizes_list, labels_list, path, color_palette='deep', size=10, label_font_size=12):
    # Validate the color palette
    if color_palette in px.colors.qualitative.__dict__:
        colors = px.colors.qualitative.__dict__[color_palette]
    else:
        colors = px.colors.qualitative.Plotly  # Default color palette

    fig = go.Figure()

    for i, (code, top_concepts, sizes, labels) in enumerate(zip(codes, top_concepts_list, sizes_list, labels_list)):
        for j, (concept, size) in enumerate(zip(top_concepts, sizes)):
            fig.add_trace(go.Bar(
                x=[code],
                y=[size],
                name=labels[j],
                marker_color=colors[j % len(colors)],
                text=labels[j],
                textposition='inside',
                hoverinfo='text',
                orientation='v'
            ))

    fig.update_layout(
        barmode='stack',
        title_text="Stacked Bar Plot for Multiple Codes",
        title_font=dict(size=18, family='Arial', color='black'),
        title_x=0.5,
        xaxis=dict(
            title='ICD9 Codes',
            tickmode='array',
            tickvals=codes,
            ticktext=codes,
            tickangle=-90
        ),
        yaxis_title='Weight',
        legend_title_text='Dictionary Feature ID',
        margin=dict(t=50, b=100, l=50, r=200),
        height=size * len(codes) * 100
    )

    # Save the figure as a static image
    # pio.write_image(fig, path)

    # Display the plot
    fig.show()
platelet_feature = 5556
top_codes, top_concepts_list, sizes_list = get_top_concepts_for_code_given_feature(global_mat, platelet_feature, icd9proc, icd9cm, label_transform, n=3)
plot_multiple_stacked_bar(top_codes, top_concepts_list, sizes_list, top_concepts_list, path="figs/platelet_related_mapping.png", color_palette="pastel", size=20)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
def get_heatmap_ICD_fids(global_mat, f_idx, icd9proc, icd9cm, L1_summaries, label_transform, n=5, top=10):
    cls_val, cls_idxs = global_mat[:,f_idx].sort(descending=True)
    top_codes = [] # first get the top codes that relate to the feature idx
    for i in range(n):
        code = label_transform.index2target[cls_idxs[i].item()]
        code_desc = ""
        if code in icd9proc:
            code_desc = icd9proc.lookup(code)
        elif code in icd9cm:
            code_desc = icd9cm.lookup(code)
        code = f"{code}: {code_desc}"
        top_codes.append(code)


    top_fids_per_ICD = set()
    # for each code compile a set of the top fids, note that our heat maps will not be square.
    for i in range(n):
        cls_f_val, cls_f_idxs = global_mat[cls_idxs[i],:].sort(descending=True)
        for j in range(top):
            top_fids_per_ICD.add(cls_f_idxs[j].item())
        #     fid = cls_f_idxs[j]
        #     if str(fid.item()) in L1_summaries:
        #         top_concepts.append(f"{fid.item()}: " + remove_common_pattern_phrases(L1_summaries[str(fid.item())]) + f", {cls_f_val[j].item():.2f}")
        #     else:
        #         top_concepts.append(f"{fid.item()}: Unidentfiable, {cls_f_val[j].item():.2f}")
        #     sizes.append(cls_f_val[j].item())
        # top_concepts_list.append(top_concepts)
        # sizes_list.append(sizes)
    top_fids_per_ICD = list(top_fids_per_ICD) # create the list so they're ordered.
    top_fid_labels = []
    for fid in top_fids_per_ICD:
        if str(fid) in L1_summaries:
            top_fid_labels.append(f"{fid}: " + remove_common_pattern_phrases(L1_summaries[str(fid)]) )
        else:
            top_fid_labels.append(f"{fid}: Unidentfiable")

    # construct heat map matrix.
    heatmap = []
    for i in range(n):
        heat_row = []
        for fid in top_fids_per_ICD:
            heat_row.append(global_mat[cls_idxs[i], fid])
        heatmap.append(heat_row)

    return heatmap, top_codes, top_fid_labels

def abbreviate_label(label, max_length=30):
    if len(label) > max_length:
        return label[:max_length] + "..."
    else:
        return label

def plot_heatmap(dictionary_features, codes, magnitudes, path, color_palette='YlOrRd', size=10):
    # Create the figure
    dictionary_features = [abbreviate_label(label) for label in dictionary_features]
    codes = [abbreviate_label(label) for label in codes]
    fig, ax = plt.subplots(figsize=(size, size * 0.8))

    # Create the heatmap
    sns.heatmap(
        magnitudes,
        xticklabels=dictionary_features,
        yticklabels=codes,
        cmap=color_palette,
        annot=True,
        cbar=True,
        square=False,
        linewidths=0.5,
        linecolor='white',
        ax=ax,
        fmt='.1f'  # Specify the format for the annotations
    )

    # Set the title and axis labels
    ax.set_title("Relationship Between Dictionary Features and Medical Codes")
    ax.set_xlabel("Dictionary Features")
    ax.set_ylabel("Medical Codes")

    # Rotate the x-axis labels
    plt.setp(ax.get_xticklabels(), rotation=90)

    # Save the figure
    plt.savefig(path, bbox_inches='tight')

    # Display the plot
    plt.show()

platelet_feature = 5556
magnitudes, codes, dictionary_features = get_heatmap_ICD_fids(global_mat, platelet_feature, icd9proc, icd9cm, L1_summaries, label_transform, n=20, top=3)
# Plot the heatmap
plot_heatmap(dictionary_features, codes, magnitudes, "figs/heatmap.png", size=20)

In [ ]:
platelet_feature = 4817
magnitudes, codes, dictionary_features = get_heatmap_ICD_fids(global_mat, platelet_feature, icd9proc, icd9cm, L1_summaries, label_transform, n=10, top=3)
# Plot the heatmap
plot_heatmap(dictionary_features, codes, magnitudes, "figs/heatmap.png", size=10)

In [ ]:
sepsis_feature = 5879
magnitudes, codes, dictionary_features = get_heatmap_ICD_fids(global_mat, sepsis_feature, icd9proc, icd9cm, L1_summaries, label_transform, n=10, top=3)
# Plot the heatmap
plot_heatmap(dictionary_features, codes, magnitudes, "figs/heatmap.png", size=10)

In [ ]:

diabetes_feature = 2353	
magnitudes, codes, dictionary_features = get_heatmap_ICD_fids(global_mat, diabetes_feature, icd9proc, icd9cm, L1_summaries, label_transform, n=10, top=3)
# Plot the heatmap
plot_heatmap(dictionary_features, codes, magnitudes, "figs/heatmap.png", size=10)

In [ ]:
# Claude Summaries ACTUALLY AMAZING lol.

additional_L1_summaries = {
    5556: "Febrile neutropenia, a common complication in cancer patients receiving chemotherapy.",
    5442: "Orthotopic liver transplantation and related procedures such as liver biopsy.",
    4443: "Trisomy disorders, likely Down syndrome (Trisomy 21), and associated genetic counseling.",
    5017: "Complications related to duodenal stent placement, perforation, and metastatic cancer.",
    5683: "Craniotomy procedures for resection of brain masses, along with supportive measures.",
    4406: "Alcohol withdrawal symptoms, including agitation, hallucinations, and complications like pancreatitis.",
    2091: "Intracranial hemorrhage, including intraparenchymal and subdural hemorrhages, and related complications.",
    2568: "Total knee and hip replacement surgeries for osteoarthritis and related conditions.",
    2501: "Complications following transjugular intrahepatic portosystemic shunt (TIPS) procedures.",
    788: "Biliary obstruction and infection, likely due to gallstones or primary sclerosing cholangitis.",
    3109: "Complications following gastrointestinal surgeries, such as gastrectomy and jejunostomy.",
    1391: "Percutaneous endoscopic gastrostomy (PEG) tube placement for nutritional support.",
    782: "Thoracoabdominal aortic aneurysm and related surgical repairs or complications.",
    5355: "Aspirin desensitization in patients with cardiovascular disease or lung cancer.",
    1584: "Intra-aortic balloon pump (IABP) placement and discontinuation for cardiac support.",
    5343: "Bleeding gastric ulcers diagnosed and treated via endoscopy and medication.",
    1713: "Feeding tube placement and tolerance, such as jejunostomy (J) tubes, in various clinical scenarios.",
    1962: "Emphysema and related respiratory conditions in patients with other comorbidities.",
    1131: "Bacterial pneumonia and its diagnosis, treatment, and associated conditions.",
    345: "Lung conditions like bronchiolitis obliterans organizing pneumonia (BOOP) and radiation pneumonitis.",
    4333: "AIDS (Acquired Immunodeficiency Syndrome) and related complications, treatments, and procedures.",
    5188: "Fractures of the scapula and glenoid fossa, often in the context of multiple traumatic injuries.",
    1530: "(Incomplete information to determine a clear pattern or concept.)",
    4780: "Metastatic cancer with spinal involvement causing pathologic fractures and related symptoms.",
    5861: "Embolization procedures for controlling bleeding from pelvic vessels, often in trauma cases.",
    4244: "Ocular conditions and related surgical interventions, such as pterygia and strabismus.",
    1558: "Neonatal hyperbilirubinemia, both physiologic and pathologic, and related diagnostic workup.",
    1775: "ST-elevation myocardial infarction (STEMI) and related cardiac catheterization procedures.",
    3329: "Sigmoid colon resection, often in the context of ischemic colitis or other abdominal emergencies.",
    2343: "(Incomplete information to determine a clear pattern or concept.)",
    802: "Patients with end-stage amyotrophic lateral sclerosis (ALS) who are ventilator-dependent and have complications such as bronchiectasis, pneumonia, and cardiac issues.",
    6069: "Patients undergoing total knee replacement surgery, particularly those with complications like respiratory distress, pain management issues, or comorbidities such as schizophrenia.",
    4917: "Patients with severe dysphagia and difficulty managing oral secretions, often in the context of advanced illnesses such as cancer or critical conditions.",
    1590: "Patients with orthopedic injuries requiring intramedullary nail placement, along with other complications such as encephalitis or cardiovascular issues.",
    5550: "Patients with anemia secondary to various conditions, such as steroid-induced myopathy, chronic diseases, or intrauterine growth restriction.",
    2457: "Patients with epilepsy, particularly those with continuous or periodic epileptiform discharges on EEG, and management with antiepileptic medications."
}

In [ ]:
def get_surrounding_heatmap_ICD_fids(global_mat, cls_idx, icd9proc, icd9cm, L1_summaries, label_transform, n=5, top=10):
    surrounding_cls_idxs = [cls_idx]
    top_codes = [] # first get the top codes that relate to the feature idx
    for i in range(n):
        code_up = cls_idx + i + 1
        code_down = cls_idx - i - 1
        surrounding_cls_idxs.append(code_up)
        surrounding_cls_idxs.append(code_down)

    surrounding_cls_idxs = sorted(surrounding_cls_idxs)
    for cls_idx in surrounding_cls_idxs:
        code = label_transform.index2target[cls_idx]
        code_desc = ""
        if code in icd9proc:
            code_desc = icd9proc.lookup(code)
        elif code in icd9cm:
            code_desc = icd9cm.lookup(code)
        code = f"{code}: {code_desc}"
        top_codes.append(code)


    top_fids_per_ICD = set()
    # for each code compile a set of the top fids, note that our heat maps will not be square.
    for cls_idx in surrounding_cls_idxs:
        cls_f_val, cls_f_idxs = global_mat[cls_idx,:].sort(descending=True)
        for j in range(top):
            top_fids_per_ICD.add(cls_f_idxs[j].item())
        #     fid = cls_f_idxs[j]
        #     if str(fid.item()) in L1_summaries:
        #         top_concepts.append(f"{fid.item()}: " + remove_common_pattern_phrases(L1_summaries[str(fid.item())]) + f", {cls_f_val[j].item():.2f}")
        #     else:
        #         top_concepts.append(f"{fid.item()}: Unidentfiable, {cls_f_val[j].item():.2f}")
        #     sizes.append(cls_f_val[j].item())
        # top_concepts_list.append(top_concepts)
        # sizes_list.append(sizes)
    top_fids_per_ICD = list(top_fids_per_ICD) # create the list so they're ordered.
    top_fid_labels = []
    for fid in top_fids_per_ICD:
        if str(fid) in L1_summaries:
            top_fid_labels.append(f"{fid}: " + remove_common_pattern_phrases(L1_summaries[str(fid)]) )
        else:
            top_fid_labels.append(f"{fid}: Unidentfiable")

    # construct heat map matrix.
    heatmap = []
    for cls_idx in surrounding_cls_idxs:
        heat_row = []
        for fid in top_fids_per_ICD:
            heat_row.append(global_mat[cls_idx, fid])
        heatmap.append(heat_row)

    return heatmap, top_codes, top_fid_labels

heatmap, codes, top_fid_labels = get_surrounding_heatmap_ICD_fids(global_mat, 3213, icd9proc, icd9cm, L1_summaries, label_transform,n=2,top=3)
plot_heatmap(top_fid_labels, codes, heatmap, "figs/platelet_heatmap.png", size=10)

In [ ]:
def plot_single_treemap(code, top_concepts, sizes, labels, path, color_palette='deep', size=60, min_text_size=12, max_text_size=24):
    # Set the figure size
    fig_width = size
    fig_height = fig_width
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))

    # Create a color palette using Seaborn
    colors = sns.color_palette(color_palette, len(sizes))

    # Wrap the labels to fit within the subsquares
    wrapped_labels = wrap_labels(labels)

    # Scale the text sizes based on the sizes
    text_sizes = scale_text_sizes(np.array(sizes), min_text_size, max_text_size)

    # Create the treemap using squarify with padding
    squarify.plot(sizes=sizes, alpha=0.8, ax=ax, color=colors, pad=True)

    # Add labels to the rectangles with scaled text sizes
    for rect, label, text_size in zip(ax.patches, wrapped_labels, text_sizes):
        x, y = rect.get_xy()
        dx, dy = rect.get_width(), rect.get_height()
        ax.text(x + dx / 2, y + dy / 2, label, fontsize=text_size, wrap=True, weight='bold', ha='center', va='center')

    # Wrap the title if it exceeds the specified number of words
    wrapped_title = wrap_title(f"ICD9 {code}", max_words=5)

    # Add the wrapped title with bold text
    ax.set_title(wrapped_title, fontsize=34, fontweight='bold')

    # Remove the axis
    ax.axis('off')

    # Set equal aspect ratio
    ax.set_aspect('equal')

    # Save the figure with tight bounding box
    plt.savefig(path, bbox_inches='tight')

    # Display the plot
    plt.show()

def get_relevant_fids(global_mat, cls_idx, n=5):
    cls_val, cls_idxs = global_mat[:,cls_idx].sort(descending=True)
    top_fids = cls_idxs[:n].tolist()
    return top_fids

def get_labels_for_code(cls_idx, L1_summaries, other_summaries, top=20):
    top_concepts = []
    sizes = []
    cls_f_val, cls_f_idxs = global_mat[cls_idx,:].sort(descending=True)
    for j in range(10):
        fid = cls_f_idxs[j]
        if str(fid.item()) in L1_summaries:
            top_concepts.append(f"{fid.item()}: " + remove_common_pattern_phrases(L1_summaries[str(fid.item())]) + f", {cls_f_val[j].item():.2f}")
        elif fid.item() in other_summaries:
            top_concepts.append(f"{fid.item()}: " + remove_common_pattern_phrases(other_summaries[fid.item()]) + f", {cls_f_val[j].item():.2f}")
        else:
            top_concepts.append(f"{fid.item()}: Unidentfiable, {cls_f_val[j].item():.2f}")
        sizes.append(cls_f_val[j].item())
    return top_concepts, sizes

fp_cls_idx = 3213
code = label_transform.index2target[fp_cls_idx]
code_desc = ""
if code in icd9proc:
    code_desc = icd9proc.lookup(code)
elif code in icd9cm:
    code_desc = icd9cm.lookup(code)
code = f"{code}: {code_desc}"
relevant_fids = get_relevant_fids(global_mat, fp_cls_idx, 20)
top_concepts, sizes = get_labels_for_code(fp_cls_idx, L1_summaries, additional_L1_summaries)

# additional_L1_summaries = 
# for fid in relevant_fids:
#     print(f"File ID: {fid}")
#     print(L1_summaries[str(fid)])
#     print(L1_contexts[fid]["context_string"])
#     print()
# relevant_fids = [5188, 4443, 345, 1558, 802, 6069, 4917] # best one lmao
plot_single_treemap(code, top_concepts, sizes, top_concepts, "figs/platelet_mapping.png", color_palette='pastel', size=5, min_text_size=34, max_text_size=46)

In [ ]:
# plot a bar plot with labels,
def plot_single_bar(code, top_concepts, sizes, labels, path, color_palette='deep', size=10, label_font_size=12):
    # Set the figure size
    fig_width = size
    fig_height = fig_width * 0.5
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))

    # Create a color palette using Seaborn
    colors = sns.color_palette(color_palette, len(sizes))

    # Create the bar plot
    bars = ax.bar(top_concepts, sizes, color=colors)

    # Add labels to the bars
    for i, label in enumerate(labels):
        bar_width = bars[i].get_width()
        bar_height = bars[i].get_height()

        # Move the text slightly up within the bar
        # text_y = bar_height * 0.5 

        text_y = 1.5

        ax.text(i, text_y, label, ha='center', va='center', fontsize=label_font_size, rotation=90, weight='bold')

    # Set the title
    wrapped_title = wrap_title(f"ICD9 {code}", max_words=5)
    ax.set_title(wrapped_title, fontsize=18, fontweight='bold')

    # Set the x-axis label
    ax.set_xlabel('Dictionary Feature ID')
    ax.set_ylabel('Weight')

    # Rotate the x-axis labels
    plt.xticks(rotation=90, visible=True)

    # Remove the top and top-right line axes
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
      # Add horizontal grid lines
    ax.yaxis.grid(True, linestyle='--', linewidth=0.5, color='gray', alpha=0.7)
    
    # Set the y-axis tick parameters
    ax.tick_params(axis='y', which='major', labelsize=12, labelcolor='black', labelrotation=0, labelright=False,
                   labelleft=True, labeltop=False, labelbottom=False, direction='out', length=5, width=1, color='black',
                   pad=5, top=False, bottom=True, left=True, right=False, zorder=10)
    plt.tight_layout()
    # Save the figure with tight bounding box
    plt.savefig(path, bbox_inches='tight')

    # Display the plot
    plt.show()

def abbreviate_label(label, max_length=30):
    if len(label) > max_length:
        return label[:max_length] + "..."
    else:
        return label

def get_labels_for_code(cls_idx, L1_summaries, other_summaries, top=20):
    top_concepts = []
    sizes = []
    fids = []
    cls_f_val, cls_f_idxs = global_mat[cls_idx,:].sort(descending=True)
    for j in range(top):
        fid = cls_f_idxs[j]
        fids.append(str(fid.item()))
        # fids.append()
        if str(fid.item()) in L1_summaries:
            top_concepts.append(remove_common_pattern_phrases(L1_summaries[str(fid.item())]))
        elif fid.item() in other_summaries:
            top_concepts.append(remove_common_pattern_phrases(other_summaries[fid.item()]))
        else:
            top_concepts.append(f"Unidentfiable, {cls_f_val[j].item():.2f}")
        top_concepts[j] = abbreviate_label(top_concepts[j])
        sizes.append(cls_f_val[j].item())
    return top_concepts, sizes, fids

relevant_fids = [5188, 4443, 345, 1558, 802, 6069, 4917, 4443, 5017, 5683, 4406, 2568]
relevant_fids = [4443, 5017, 5683, 4406, 2568]
top_concepts, sizes, fids = get_labels_for_code(fp_cls_idx, L1_summaries, additional_L1_summaries, top=30)

plot_single_bar(code, fids, sizes, top_concepts,path="figs/platelet_bar.png", color_palette="pastel", size=20)

In [ ]:
print(top_concepts)

In [ ]:
import plotly
import plotly.express as px
import plotly.graph_objects as go
import importlib
importlib.reload(plotly)
importlib.reload(px)
importlib.reload(go)
import kaleido
importlib.reload(kaleido)
def plot_single_bar(code, top_concepts, sizes, labels, path, color_palette='deep', size=10, label_font_size=12):
    # Validate the color palette
    if color_palette in px.colors.qualitative.__dict__:
        colors = px.colors.qualitative.__dict__[color_palette]
    else:
        colors = px.colors.qualitative.Plotly  # Default color palette

    # Ensure the number of colors matches the number of bars
    if len(colors) < len(sizes):
        colors = colors * (len(sizes) // len(colors) + 1)
    
    # Create the bar plot
    fig = px.bar(x=top_concepts, y=sizes, color=top_concepts, color_discrete_sequence=colors[:len(sizes)])

    # Add custom text labels to the bars
    for i, (size, label) in enumerate(zip(sizes, labels)):
        fig.add_annotation(
            x=top_concepts[i],
            y=size,
            text=label,
            showarrow=False,
            yshift=15,
            font=dict(size=label_font_size, color='black', family='Arial')
        )

    # Set the title
    fig.update_layout(
        title_text=f"ICD9 {code}",
        title_font=dict(size=18, family='Arial', color='black'),
        title_x=0.5,
        xaxis_title='Dictionary Feature ID',
        yaxis_title='Weight',
        xaxis_tickangle=-90,
        showlegend=False,
        margin=dict(t=50, b=100, l=50, r=50)
    )

    # Save the figure as a static image
    # fig.write_image(path)

    # Display the plot
    fig.show()

# Example usage
code = "123"
top_concepts = ['Concept1', 'Concept2', 'Concept3', 'Concept4', 'Concept5']
sizes = [10, 15, 7, 10, 20]
labels = ['Label1', 'Label2', 'Label3', 'Label4', 'Label5']
path = "bar_plot.png"

plot_single_bar(code, top_concepts, sizes, labels, path)


In [ ]:
plot_single_bar(code, fids, sizes, top_concepts,path="figs/platelet_bar.png", color_palette="pastel")